# Contents of the file.py:

- Working the data with Python
- Export the data cleaned into a CSV file to import them into MySQL
- MySQL queries to get the positive and negative insights based on keywords

# Working the data with Python:

## Import Python packages:

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import pymysql
from sqlalchemy import create_engine
import getpass                          # password hiding tool
from IPython.display import display

## Convert TSV file to CSV:

In [2]:
tsv_file = 'datasetTwitter.tsv'
csv_file = pd.read_table(tsv_file, sep = '\t')
csv_file.to_csv('data.csv', index = False)

In [3]:
data = pd.read_csv('data.csv')

## Data Cleaning:

In [4]:
# Converting the header names to lowercase:

cols = []

for column in range(len(data.columns)):
    cols.append(data.columns[column].lower())

data.columns = cols

# Replacing ' ' to '_':

data.columns = data.columns.str.replace(' ', '_')

## Checking the % of null values in each column:

In [5]:
nulls_test = pd.DataFrame(round(data.isna().sum() / len(data), 4) * 100)
nulls_test = nulls_test.rename(columns={'index':'header', 0: '% nulls'})
nulls_test

,% nulls
modularity,5.56
eigenvector,5.56
no._of_comments,0.00
no._of_likes,0.00
no._of_shares,0.00
no._of_retweets,0.00
content,0.00
bio,3.56
search,0.00
source,0.00


## Dropping whole columns without data:

In [6]:
data2 = data.drop(['title', 'image_tags', 'tags', 'main_emotion', 'emotions', 'erep'], axis = 1)

In [7]:
data2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200000 entries, 0 to 199999
Data columns (total 36 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   modularity                188886 non-null  float64
 1   eigenvector               188886 non-null  float64
 2   no._of_comments           200000 non-null  int64  
 3   no._of_likes              200000 non-null  int64  
 4   no._of_shares             200000 non-null  int64  
 5   no._of_retweets           200000 non-null  int64  
 6   content                   200000 non-null  object 
 7   bio                       192887 non-null  object 
 8   search                    200000 non-null  int64  
 9   source                    200000 non-null  object 
 10  application               200000 non-null  object 
 11  date                      200000 non-null  int64  
 12  parent                    200000 non-null  object 
 13  language                  199907 non-null  o

In [8]:
pd.set_option('display.max_columns', None)

In [9]:
data2.head(1)

,modularity,eigenvector,no._of_comments,no._of_likes,no._of_shares,no._of_retweets,content,bio,search,source,application,date,parent,language,parent_source_identifier,domain,topics,sentiment,sentiment_class,sentiment_by,visibility,potential_impressions,actual_impressions,city,country,latitude,longitude,user_city,user_country,user_latitude,user_longitude,no._of_followers,no._of_friends,gender,links_url,post_subtype
0,1.0,0.109824,0,2990,0,448,Paying for sex was the lowest moment in my sex...,"GIGO. | d(-_-)b | 6 stars = GC, BH, SG... DY A...",79684,Twitter,Twitter for Android,1621538636000,0,tl,id:,twitter.com,"Niliendanga Latema na psyche,Kitu sikuwahi ele...",-0.487021,negative,1,1812.19,15618,1717,NaN,KE,-0.023559,37.906193,NaN,KE,-0.023559,37.906193,15618,1093,unknown,NaN,original post


In [10]:
data2.describe()

,modularity,eigenvector,no._of_comments,no._of_likes,no._of_shares,no._of_retweets,search,date,sentiment,sentiment_by,visibility,potential_impressions,actual_impressions,latitude,longitude,user_latitude,user_longitude,no._of_followers,no._of_friends
count,188886.000000,188886.000000,200000.0,200000.000000,200000.0,200000.000000,200000.0,2.000000e+05,189412.000000,200000.0,200000.000000,2.000000e+05,200000.000000,194647.000000,194647.000000,194647.000000,194647.000000,2.000000e+05,200000.000000
mean,6.647242,0.032452,0.0,1.859475,0.0,0.473855,79684.0,1.621625e+12,-0.128341,1.0,35.844280,1.793590e+04,1972.462570,-0.184073,37.738311,-0.183987,37.738358,1.793590e+04,2859.476120
std,4.428636,0.047646,0.0,23.048289,0.0,3.182747,0.0,6.253084e+09,0.625219,0.0,34.642699,1.643920e+05,18083.122348,0.857525,1.381431,0.857434,1.381405,1.643920e+05,6586.961872
min,0.000000,0.000000,0.0,0.000000,0.0,0.000000,79684.0,1.609503e+12,-0.999956,1.0,1.000000,1.000000e+00,0.000000,-82.862752,-169.315421,-82.862752,-169.315421,0.000000e+00,0.000000
25%,4.000000,0.000000,0.0,0.000000,0.0,0.000000,79684.0,1.616233e+12,-0.725991,1.0,20.440000,5.980000e+02,65.000000,-0.023559,37.906193,-0.023559,37.906193,5.980000e+02,470.000000
50%,6.000000,0.003259,0.0,0.000000,0.0,0.000000,79684.0,1.622121e+12,0.000000,1.0,30.230000,1.655000e+03,182.000000,-0.023559,37.906193,-0.023559,37.906193,1.655000e+03,1114.000000
75%,8.000000,0.050223,0.0,0.000000,0.0,0.000000,79684.0,1.626853e+12,0.454184,1.0,41.620000,4.628000e+03,509.000000,-0.023559,37.906193,-0.023559,37.906193,4.628000e+03,2704.000000
max,40.000000,0.304044,0.0,2990.000000,0.0,448.000000,79684.0,1.632002e+12,0.999581,1.0,2920.080000,3.804083e+06,418449.000000,56.952041,86.416500,56.952041,86.416500,3.804083e+06,137312.000000


## Counting distinct words:

In [11]:
from collections import Counter

In [12]:
words = Counter(" ".join(data2['content']).split(" ")).items()

In [54]:
# words

In [14]:
list_ordered = list(words)

In [15]:
# list_ordered.sort(key=lambda list_ordered: list_ordered[1], reverse=True)

Printing 'list_ordered' we would get the words with more repetitions at the top positions. Therefore, we could check which words have had more impact in this matter.

Unfortunately, the program crash after executing the print function because of the list's dimension.

# Export the data cleaned into a CSV file to import them into MySQL

## Export CSV file:

In [40]:
data2.to_csv('data1.csv')

## MySQL Connection and query testing:

This is the script I created to create the database to import the data (please double click for a better display):

drop database if exists twitter;
create database twitter;
use twitter;

create table srh(
	id int,
	modularity decimal(3, 1),
	eigenvector decimal(20, 15),
	num_comments int,
	num_likes int,
	num_shares int,
	num_retweets int,
	content varchar(500),
	bio varchar(300),
	search int,
	source varchar(100),
	application varchar(100),
	date_t bigint,
	parent varchar(100),
	lang varchar(20),
	par_id_source varchar(70),
	domain_t varchar(70),
	topics varchar(250),
	sentiment decimal(10, 8),
	sent_class varchar(15),
	sentiment_by int,
	visibility decimal(12, 8),
	pot_impressions int,
	act_impressions int,
	city varchar(100),
	country varchar(100),
	latitude decimal(15, 10),
	longitude decimal(15, 10),
	user_city varchar(100),
	user_country varchar(100),
	user_lat decimal(15, 10),
	user_long decimal(15, 10),
	num_fol int,
	num_friends int,
	gender varchar(20),
	links_url varchar(200),
	post_subtype varchar(100)
) engine=innoDB default charset=utf8;

load data local infile 'data1.csv' into table srh character set latin1 fields terminated by ',' enclosed by '"' ignore 1 lines;

In [16]:
mysql_pw = getpass.getpass()

········


In [17]:
connection_string = 'mysql+pymysql://lbadia:' + mysql_pw + '@127.0.0.1:3306/twitter'
engine = create_engine(connection_string)

In [18]:
testing = pd.read_sql_query('Select * from srh limit 1', engine)
testing

,id,modularity,eigenvector,num_comments,num_likes,num_shares,num_retweets,content,bio,search,source,application,date_t,parent,lang,par_id_source,domain_t,topics,sentiment,sent_class,sentiment_by,visibility,pot_impressions,act_impressions,city,country,latitude,longitude,user_city,user_country,user_lat,user_long,num_fol,num_friends,gender,links_url,post_subtype
0,0,1.0,0.109824,0,2990,0,448,Paying for sex was the lowest moment in my sex...,"GIGO. | d(-_-)b | 6 stars = GC, BH, SG... DY A...",79684,Twitter,Twitter for Android,1621538636000,0,tl,id:,twitter.com,"Niliendanga Latema na psyche,Kitu sikuwahi ele...",-0.487021,negative,1,1812.19,15618,1717,,KE,-0.023559,37.906193,,KE,-0.023559,37.906193,15618,1093,unknown,,original post\r


# MySQL queries to get the positive and negative insights based on keywords:

## Tweets with more likes (top 20):

In [19]:
pd.set_option('display.max_colwidth', None)

In [20]:
max_likes = pd.read_sql_query('select num_likes, content, topics, sentiment from srh order by num_likes desc limit 20;', engine)
max_likes

,num_likes,content,topics,sentiment
0,2990,"Paying for sex was the lowest moment in my sex life.Niliendanga Latema na psyche I imagined al even grab boobies, spank her nshit. Kulipa 150 bob nilivalishwa condom haraka na mti ikavurutwa ikaingizwa ndani. Kitu sikuwahi elewa was why pussy ya mapoko would always be so hot!","Niliendanga Latema na psyche,Kitu sikuwahi elewa,lowest moment,boobies,pussy ya mapoko,sex life,Kulipa,bob nilivalishwa condom haraka na mti ikavurutwa ikaingizwa ndani,nshit",-0.487021
1,2905,"Niko hapa kuwakumbusha huku nje watu ni wagonjwa, trust no one, normalize using condom and sticking to one partner","Niko hapa kuwakumbusha huku nje watu ni wagonjwa,condom,partner",0.000000
2,2891,"If condom isn't available, what else can one use as protection?","condom,protection",0.000000
3,2374,There is currently a shortage of ARVs and condoms in the country. How you choose to use this information is entirely up to you. Enjoy your week.,"shortage of ARVs,condoms,country,information",0.000000
4,1938,"Enough with the sex education, how about you teach us how to make 10 grand a week without leaving the house?","sex education,house,grand",0.354187
5,1934,Go off then Contraceptive Captain. Regal us oh Cervical Caesar. Teach us your ways Fallopian Ipupa.. https://t.co/1VezWhPkyV,"ways Fallopian Ipupa,Contraceptive Captain,Cervical Caesar",0.569425
6,1868,"Let me tell you why I love Catholics because it's very hard to find them arguing over trivial issues, the pope says don't use contraception, they say amen and apply common sense ..they buy contraceptives","trivial issues,common sense,Catholics,pope,contraception,contraceptives",0.618377
7,1719,"If you ever advise someone you impregnated by mistake to do an abortion and she declines, she should raise the kid ALONE! -the end-","mistake,abortion,kid,end",-0.822875
8,1528,"We have not forgotten how you bought one condom for Ksh 5,000 when you were serving as devolution CS. https://t.co/of4dYWhWqz","devolution CS,condom,Ksh",-0.941069
9,1407,"120, 000 condoms were used during the WRC event in Naivasha, indeed the Subaru guys 'won' the race by far.","WRC event,condoms,Subaru guys,race,Naivasha",0.000000


As I can see, most of the tweets with the most amount of likes have negative connotations. Therefore, it has relation to the sentiment average (negative). Some of them are sexist and others reflect that the population has a lack of information about contraceptive methods.
														
It seems that the most controversial ones are the ones with more acceptance among the users.

## Tweets with less likes (top 20):

In [21]:
min_likes = pd.read_sql_query('select num_likes, content, topics, sentiment from srh order by num_likes limit 20;', engine)
min_likes

,num_likes,content,topics,sentiment
0,0,"Girls who undergo FGM face short-term complications such as severe pain, shock, bleeding, infections, and difficulty in passing urine, as well as long-term consequences for their sexual, reproductive health and mental health.#GirlsGetEqual https://t.co/OK4TT6GUT9","FGM face short-term complications,severe pain,Girls,reproductive health,long-term consequences,mental health,shock,infections,difficulty,urine,#GirlsGetEqual",-0.951816
1,0,FGM is an archaic cultural practice that assaults the wellbeing of individuals and society and pitched for its complete eradication#GirlsGetEqual https://t.co/wUTb1C3sMu,"archaic cultural practice,FGM,wellbeing of individuals,society,complete eradication #GirlsGetEqual",-0.955708
2,0,"In a World Bank survey on young people's health and financial status in Kenya, it was found that most young people are worried about contracting AIDS, getting early pregnancies, getting physically or sexually exploited. #TujulishaneOnMaternalHealth","World Bank survey,financial status,young people's health,young people,Kenya,AIDS,early pregnancies,TujulishaneOnMaternalHealth",-0.907789
3,0,"WHO (2008) estimated that more than 130 million girls and women alive then had undergone FGM/C, primarily in Africa and, to a lesser extent, in some countries in the Middle East. The number has not steadily gone down today. #MediaToEndFGM","lesser extent,Middle East,girls,women,FGM,number,Africa,countries,today",-0.383718
4,0,"The boys have not been left behind as they are being taught about menstruation, sexual reproductive health as well as life skills. Also, 57 community based mentors will be trained to support the adolescents as they go through life #TwendeTukiuke https://t.co/TulonZqrR9","life skills,sexual reproductive health,boys,community,mentors,menstruation,adolescents,life #TwendeTukiuke",0.742495
5,0,It is crucial that the culprits are arrested and handed heavy fines to halt the increasing cases of FGM. #MediaToEndFGM,"heavy fines,culprits,increasing cases of FGM",-0.852744
6,0,RT @Dreymwangi: Ukipata condoms kwa boyfriend yako doesn't warrant you to become mad at him.,"RT @Dreymwangi,Ukipata condoms kwa boyfriend yako",-0.643405
7,0,@PSIimpact Partnership to allow unrestricted funding to youth-led &amp; youth-serving organizations to allow for diverse &amp; progressive formulation of programs that are able to design Adolescent &amp; Youth Sexual Reproductive Health #AYSRH @FP2030Global @WomenDeliver @BeGirlOrg #WeChoose #IYD2021,"@PSIimpact Partnership,youth-serving organizations,progressive formulation of programs,unrestricted funding,youth,Youth Sexual Reproductive Health #AYSRH @FP2030Global @WomenDeliver @BeGirlOrg #WeChoose,Adolescent",0.723775
8,0,"A key action point of the Action Coalition on Bodily Autonomy &amp; SRHR is to increase delivery of comprehensive sexuality education in &amp; out of school reaching 50 million more children, adolescents, and youth by 2026. Register for the forum here: https://t.co/ulmURGzJUq #Youth4GEF","key action point of the Action Coalition,delivery of comprehensive sexuality education,Bodily Autonomy,school,children,forum,adolescents,youth,SRHR,Register,https://t.co/ulmURGzJUq",0.000000
9,0,"@PSIimpact Ensuring that health care workers are also trained on self care options to ensure that sensitization of this is shared to adolescents &amp; young people, working with gvments to further understand the link of SRHR &amp; #AYSRH @FP2030Global @WomenDeliver @BeGirlOrg #WeChoose #IYD2021","health care workers,self care options,young people,sensitization,adolescents,link of SRHR,#AYSRH @FP2030Global @WomenDeliver @BeGirlOrg #WeChoose,gvments",0.789724


As I can analyse, a large part of the tweets that have no acceptance is the ones demanding recognition and solutions to these endemic problems. For example, better education, funding, treatment and bringing to justice the culprits. Therefore, governments must give responses to tackle those problems and give safeness to the population.

In addition, most of the tweets analysed have a high percentage of polarity (positive and negative sentiment).

## Tweets with more retweets (top 20):

In [22]:
max_ret = pd.read_sql_query('select num_retweets, num_likes, content, topics, sentiment from srh order by num_retweets desc limit 20;', engine)
max_ret

,num_retweets,num_likes,content,topics,sentiment
0,448,2990,"Paying for sex was the lowest moment in my sex life.Niliendanga Latema na psyche I imagined al even grab boobies, spank her nshit. Kulipa 150 bob nilivalishwa condom haraka na mti ikavurutwa ikaingizwa ndani. Kitu sikuwahi elewa was why pussy ya mapoko would always be so hot!","Niliendanga Latema na psyche,Kitu sikuwahi elewa,lowest moment,boobies,pussy ya mapoko,sex life,Kulipa,bob nilivalishwa condom haraka na mti ikavurutwa ikaingizwa ndani,nshit",-0.487021
1,285,2905,"Niko hapa kuwakumbusha huku nje watu ni wagonjwa, trust no one, normalize using condom and sticking to one partner","Niko hapa kuwakumbusha huku nje watu ni wagonjwa,condom,partner",0.000000
2,279,936,Government is on the spot for releasing expired family planning implants to Kenyan womenhttps://t.co/MJoV8QGbEo https://t.co/rWr45dGm9Y,"family planning implants,Government,spot,Kenyan women",0.000000
3,270,2374,There is currently a shortage of ARVs and condoms in the country. How you choose to use this information is entirely up to you. Enjoy your week.,"shortage of ARVs,condoms,country,information",0.000000
4,249,977,"1973: Mwai Kibaki eloquence as a minister of Finance on matters family planning in Vihiga. No offense but honestly, nowadays if a journalist pose such a question to the likes of DP William Ruto, President Uhuru Kenyatta or Raila Odinga, they will fumble with no clear answer. https://t.co/KL2X4VNUxg","Mwai Kibaki eloquence,likes of DP William Ruto,journalist pose,Raila Odinga,offense,President Uhuru Kenyatta,question,minister of Finance,matters family planning,clear answer,nowadays,Vihiga",0.557522
5,242,1868,"Let me tell you why I love Catholics because it's very hard to find them arguing over trivial issues, the pope says don't use contraception, they say amen and apply common sense ..they buy contraceptives","trivial issues,common sense,Catholics,pope,contraception,contraceptives",0.618377
6,223,1934,Go off then Contraceptive Captain. Regal us oh Cervical Caesar. Teach us your ways Fallopian Ipupa.. https://t.co/1VezWhPkyV,"ways Fallopian Ipupa,Contraceptive Captain,Cervical Caesar",0.569425
7,221,1528,"We have not forgotten how you bought one condom for Ksh 5,000 when you were serving as devolution CS. https://t.co/of4dYWhWqz","devolution CS,condom,Ksh",-0.941069
8,165,1308,"Women use tradition to try to manipulate men into servanthood, but don't want to conform to the same traditions that return then to the kitchen and submissiveness. If she can't do for you everything she expects you to do for her, abort mission. ""Real man"" card ONLY benefits them","Real man,Women,tradition,traditions,card,kitchen,mission,servanthood,submissiveness",0.247026
9,162,1232,"I will vote for whoever, makes abortion accessible, safe and legal in Kenya.","abortion,Kenya",0.829364


Some of the tweets with more retweets are the ones with more likes (with negative connotations). However, some of them reclaim solutions and improvements in the health system (positive sentiment).

## Tweets with less retweets (top 20):

In [23]:
min_ret = pd.read_sql_query('select num_retweets, content, topics, sentiment from srh order by num_retweets limit 20;', engine)
min_ret

,num_retweets,content,topics,sentiment
0,0,@13ellfield @StormFTUTD @Vintage_Utd @utd_mason @RealistGlizzy One condom could a have stopped this comment from appearing,"@13ellfield,condom,@utd_mason",-0.702190
1,0,@its_wanja Condoms na contraceptives,@its_wanja Condoms na contraceptives,0.000000
2,0,@Tajirijack @HellenBen2 @NCBABankKenya @ncbacare @JohnGachora Someone should have used a condom!! https://t.co/XgMbivUZnv,"@Tajirijack @HellenBen2 @NCBABankKenya @ncbacare @JohnGachora,condom",-0.671636
3,0,RT @Dreymwangi: Ukipata condoms kwa boyfriend yako doesn't warrant you to become mad at him.,"RT @Dreymwangi,Ukipata condoms kwa boyfriend yako",-0.643405
4,0,"@_fels1 show up in Lolwe I show you how they make money!!I deliver condoms sometimes, lakini wamekaa bila kunicall buana","lakini wamekaa bila kunicall buana,condoms,money,Lolwe",-0.391158
5,0,"@AshaRangappa_ What happened to ""my body my choice""? Does it only hold when discussing abortions?","body,choice,abortions",-0.866922
6,0,@Oloni Nobody's is worried about STDs?,"@Oloni Nobody,STDs",-0.910709
7,0,Ukifanya sana abortion huwezi kuelewa maana ya mama lakini aliyebeba mimba na kuzaa kwa uchungu anaelewa na anajivunia kuitwa Mama.,,0.000000
8,0,@Iam_arryanne Heri ivo nilikufinger na condomðŸ˜ðŸ˜ðŸ˜,,0.000000
9,0,Mwenge wa uhuru unalala huku mbeya leo serikali inagawa condoms Kwa vijana kwani inatuchukuliaje Sisi ðŸ¤” https://t.co/mQi0svtxLW,,0.000000


In almost all of the tweets analysed appear the word "condom". Some of them try to explain facts and how is the situation in Kenya (but it could be transported to several countries in Africa).

## Key words:

Brief analyse of how many times a list of words appear in the dataset. These are the words chosen to have a strong significance in the field. Unfortunately, because it is not possible to check the words with more appearances (the program crashes), therefore, they have been selected looking in the column 'topics' or I think are relevant for the topic to get insights:

- condom
- abortion
- rape
- justice
- freedom
- human right(s)
- health
- choice
- contraceptive
- empower (empowerment)
- politic
- violen (violent, violence)
- hiv
- pregnan (pregnancy, pregnant)


Before jumping into the keywords, doing a subsequent conclusion is that an important proportion of tweets with strong positive sentiment are related to demanding and improving human rights. Perhaps the sentiment analysis is not very accurate in those cases.

### Condom

In [38]:
condom = pd.read_sql_query('select count(num_retweets) as "count_condom" from srh where content regexp "condom";', engine)
condom

,count_condom
0,37584


In [113]:
condom_tw_pos = pd.read_sql_query('select distinct(content), sentiment from srh where content regexp "condom" order by sentiment desc limit 10;', engine)
condom_tw_pos

,content,sentiment
0,"Such a beautiful and cozy weather for a Saturday. Stay gorgeous, safe and use a condom â™¥ï¸",0.995322
1,"@mutami_susan Wow this is a good initiative may God bless you...sanitary wear needs to be free, why is it condoms are made easy available but no sanitary wearðŸ¤¦â€â™€ï¸",0.993704
2,"As we welcome the new month and enjoy the happiness it brings, let us not forget to enjoy the month safely.Grab your LifeGuard condoms today from @MyDawaApp by clicking the link below:https://t.co/jvkpDqGStA#LiveLifeGuarded https://t.co/nxGfCQEKce",0.992847
3,"#CondomDay2021 in Nairobi, Kenya was ðŸ”¥ ðŸ”¥ ðŸ”¥!Condoms were distributed and information on proper condom use widely shared. Thank you everyone who participated! @CSA_Kenya @KenyaSRHR @NairobiYac @NMS_Kenya @ahfkenya @NACC_Kenya#SafeisSexy #SaferIsFun https://t.co/Ea87IAGed3",0.991282
4,"RT @StraightTalkKe: #CondomDay2021 in Nairobi, Kenya was ðŸ”¥ ðŸ”¥ ðŸ”¥!Condoms were distributed and information on proper condom use widely shared. Thank you everyone who participated! @CSA_Kenya @KenyaSRHR @NairobiYac @NMS_Kenya @ahfkenya @NACC_Kenya#SafeisSexy #SaferIsFun https://t.co/Ea87IAGed3",0.991282
5,"With love in the air, flushy candle lights, a well set condom pack, perfectly set wine glasses and a comfy king-size bed, which is the best music genre to sweeten the date?Tell us under the comment sections below!#LiveLifeGuarded https://t.co/wVEwZ87dlO",0.988322
6,RT @AishaPatricia4: Thank you Lord for another brand new day. Good morning my good friends â™¥ï¸Stay positive Have sex but don't forget to use a condom.,0.986007
7,"We bet you are already smelling the strawberry ðŸ“ scent from the window ðŸ˜‰Get the Strawberry scented Kiss Condom, it's scented &amp; smooth to pleasure your partner's senses and increases the excitement for you and your partner. Got it for only RRP Kshs 60#KissStrawberry https://t.co/GhlBnco2cr",0.984075
8,"Valentine is a very special day, for love is in the air. How about safety love with #Condom? That was the main message from our team to the people along the beaches of Suba during #InternationalCondomDay #Dual_Protection https://t.co/QPjFQoJtTb",0.983965
9,"RT @Kulah_Nzomo: Be ready for the new story anytime from now coz tommorow is world radio day with a good theme New world new Radio, Also it's an international condom day with a theme of safer is fun@otoidamian @SFCG_Kenya @KECOSCE @Amanicentercbo @ctuckwood @johngreenotunga",0.979197


In [114]:
condom_tw_neg = pd.read_sql_query('select distinct(content), sentiment from srh where content regexp "condom" order by sentiment limit 10;', engine)
condom_tw_neg

,content,sentiment
0,RT @AfyaJadili: @Nimechanuka @Yasleaders @AmplifyTweets @Zamara_fdn @NenaNaBinti @NairobiYac @trippleklive @KenyaSRHR @_Itaeli @rhnkorg @NAYAKenya Reasons why a condom may fail: 1) expired 2)poor storage conditions 3) failure to check whether a condom is intact before use 4) poor technique when wearing 5) Poor technique when removing 6)Pre-ejaculate from genital contact before wearing a condom,-0.999341
1,@Nimechanuka @Yasleaders @AmplifyTweets @Zamara_fdn @NenaNaBinti @NairobiYac @trippleklive @KenyaSRHR @_Itaeli @rhnkorg @NAYAKenya Reasons why a condom may fail: 1) expired 2)poor storage conditions 3) failure to check whether a condom is intact before use 4) poor technique when wearing 5) Poor technique when removing 6)Pre-ejaculate from genital contact before wearing a condom,-0.999011
2,Fucked a niggers chic for new year amd dude called me to confirm worse off he even gave her transport money and bitch bought condoms to be safe,-0.999005
3,@NAYAKenya @rhnkorg @ywli_info @AYARHEP_KENYA @StraightTalkKe @YourAuntyJane @MADACI_ke @NenaNaBinti @FP2030Global @KombeMartha @Alvinmwangi254 @GirlsWomenPower @esther_aoko @Eve_odhis @YouthActKE @TICAH_KE @AbukaAlfred @MokayaKelvinOb @DreamAchieversk #SafeSex means engaging in sexual relations through use of a condom which reduces the risk of unintended pregnancy and transmission of sexually transmitted infections (STIs) #WhenSexHurts #FP4ALL #NAYAVoices @ReproRights @FP2030Global @NAYAKenya @,-0.998955
4,"RT @CapitalFMKenya: @VNjoroge @PeterNgure @dngkenya @MozDJ @better4kenya @dada_sphere @PunguzaKE @kubambatv @kubambaradio @PublicPathways @MarigaThoithi We spoke about condomsâ€¦like ""if you use a condom, it's not the same"" Our fear was never STIs, our biggest fear was impregnating a girl - @mozdj#ChezaKamaWewe #Better4Kenya",-0.998954
5,"@VNjoroge @PeterNgure @dngkenya @MozDJ @better4kenya @dada_sphere @PunguzaKE @kubambatv @kubambaradio @PublicPathways @MarigaThoithi We spoke about condomsâ€¦like ""if you use a condom, it's not the same"" Our fear was never STIs, our biggest fear was impregnating a girl - @mozdj#ChezaKamaWewe #Better4Kenya",-0.998546
6,@ntvkenya @Karanjakibicho is one stenchy repugnant smelly busy who knows nothing about the presidency which has the president nd the Dp so let you itchy ass cool you only got 10 months in office nd we gonna fuck your stenchy ass without. A condom,-0.998060
7,@JunetMohamed You are the reason why a Condom was invented...To prevent such useless minds from seeing the day.I just pitty the people who woke up to vote this USELESS leaders. it's a shame,-0.997958
8,RT @AnnitahRaey: To those grown us men sleeping with kids...Wear a fucking Condom...Fucking pieces of shit..,-0.997956
9,RT @beverly_nkirote: @NAYAKenya @rhnkorg @ywli_info @AYARHEP_KENYA @StraightTalkKe @YourAuntyJane @MADACI_ke @NenaNaBinti @FP2030Global @KombeMartha @Alvinmwangi254 @GirlsWomenPower @esther_aoko @Eve_odhis @YouthActKE @TICAH_KE @AbukaAlfred @MokayaKelvinOb @DreamAchieversk #SafeSex means engaging in sexual relations through use of a condom which reduces the risk of unintended pregnancy and transmission of sexually transmitted infections (STIs) #WhenSexHurts #FP4ALL #NAYAVoices @ReproRights @FP20,-0.997438


- Positive: Festivity commemorations. Enjoy the day. Advise using the condom while doing sex.


- Negative: Bad conditions of the condom or bad sex techniques. Racist vocabulary. Empowerment to reduce the risk of pregnancy.

### Abortion

In [39]:
abortion = pd.read_sql_query('select count(num_retweets) as "count_abortion" from srh where content regexp "abortion";', engine)
abortion

,count_abortion
0,29936


In [115]:
abortion_tw_pos = pd.read_sql_query('select distinct(content), sentiment from srh where content regexp "abortion" order by sentiment desc limit 10;', engine)
abortion_tw_pos

,content,sentiment
0,"VarietyÂ callsÂ Never Rarely Sometimes AlwaysÂ ""a quietly devastating gem"";Â IndiewireÂ slammed its Oscar snub as ""a crime"";Â andÂ Rolling StoneÂ says, ""Eliza Hittman's devastating abortion drama, starring a wonder of an actress named Sidney Flanigan, is an emotional wipeout that'sâ€¦",0.996020
1,RT @IpasOrg: Now more than ever we're feeling the love for abortion providers â¤ï¸ðŸ’™ ðŸ’œ ðŸ’› ðŸ’š ðŸ§¡ Healthcare workers have gone above and beyond for us and we are so grateful for all that they do! #SRHR #reprorights #reprojustice https://t.co/oz977L7YkJ,0.995061
2,Benefits of a proper nutrition in beans1. Establishes a good early growth2. Boosts yield and crop productivity3. Ensures a continued strong crop growth4. Minimizes flower and flower bud abortion5. Boosts plant's vigor6. Improves the quality of produce #Agriculture https://t.co/RQRWUEfV0H,0.994540
3,RT @ReproRights: #Gibraltar: A referendum to liberalize one of the LAST remaining abortion bans in Europe was just backed by big majority. Huge congratulations to @GibraltarforYes and everyone who supported this vote for #reprorights! https://t.co/W2NWxeCtcJ,0.991353
4,"RT @safe2choose: Give love and receive love. Abortions are a form of self love, we love and support all women who are seeking information and services about safe abortion. https://t.co/8tb60yPd5R",0.991149
5,"RT @SAAFfund: Today we thank and celebrate midwives, providing reproductive health care all over the world! ðŸŒŽThis image shows SAAF grantee partner @mabiaghana training student midwives on safe abortion care in Ghana ðŸ‡¬ðŸ‡­#IDM #IDM2021 #InternationalDayoftheMidwife https://t.co/PEjqm16V52",0.989186
6,"RT @SAAFfund: SAAF is proud to be a signatory on this letter urging the UK government to continue the option of abortion by telemedicine #PillsByPostRemote access to abortion medication has proven to be safe, effective, and well liked by patients. https://t.co/0Bk3BK2RXq",0.987413
7,RT @SAAFfund: We got so excited about Argentina's abortion rights victory inspiring others around the world that we made a meme about it! ðŸ’š https://t.co/LJK8ImvnCn,0.986008
8,I'm very proud life. So any news about abortion ban is good news to me. Kudos to Texas ðŸ‘,0.985830
9,"Thank you @rhnkorg for always highlighting the elephant in the room, early pregnancies have a negative effect on young girls holistic wellness to their 2nd generation. #teenpregnancyke #SafeAbortionKE https://t.co/fORYib8vUF",0.984635


In [116]:
abortion_tw_neg = pd.read_sql_query('select distinct(content), sentiment from srh where content regexp "abortion" order by sentiment limit 10;', engine)
abortion_tw_neg

,content,sentiment
0,@Liz_Wheeler So Israel killed the babies....Sad...you can't be that sad. Human shields from a bomb...Sad...Fact is and it can't be changed. Israel has killed more babies than abortion.,-0.999956
1,"RT @maita_shelmith: @NAYAKenya @NMS_Kenya @MADACI_ke @lovemafrica @AYARHEP_KENYA @YouthActKE @rhnkorg @StraightTalkKe @ywli_info @CSA_Kenya @TICAH_KE @YourAuntyJane When @NMS_Kenya fails to prioritize SRHR and budgetary allocations for SRH ,it encourages the continuation of quack abortion services,increases the alarming numbers of unwanted teen pregnancies ,unsafe abortions and even death .#TheBudgetWeWant #BudgetForReproductiveHealth",-0.999942
2,"@NAYAKenya @NMS_Kenya @MADACI_ke @lovemafrica @AYARHEP_KENYA @YouthActKE @rhnkorg @StraightTalkKe @ywli_info @CSA_Kenya @TICAH_KE @YourAuntyJane When @NMS_Kenya fails to prioritize SRHR and budgetary allocations for SRH ,it encourages the continuation of quack abortion services,increases the alarming numbers of unwanted teen pregnancies ,unsafe abortions and even death .#TheBudgetWeWant #BudgetForReproductiveHealth",-0.999906
3,"RT @MADACI_ke: @NAYAKenya @NMS_Kenya @lovemafrica @AYARHEP_KENYA @YouthActKE @rhnkorg @StraightTalkKe @ywli_info @CSA_Kenya @TICAH_KE @YourAuntyJane Q4:When NMS fails to #BudgetForReproductiveHealth women and girls will be denied access to Sexual and Reproductive Health services and information hence putting the at risk of teenage pregnancies, unsafe abortions and STIs and HIV infections.#TheBudgetWeWant",-0.999846
4,"@NAYAKenya @NMS_Kenya @lovemafrica @AYARHEP_KENYA @YouthActKE @rhnkorg @StraightTalkKe @ywli_info @CSA_Kenya @TICAH_KE @YourAuntyJane Q4:When NMS fails to #BudgetForReproductiveHealth women and girls will be denied access to Sexual and Reproductive Health services and information hence putting the at risk of teenage pregnancies, unsafe abortions and STIs and HIV infections.#TheBudgetWeWant",-0.999790
5,@RealCandaceO Unfortunately in today's society;Materialism is idolised Immorality is glamorisedTruth is minimisedSin is normalisedGod is marginalised Christians demonisedFaith is secularisedSex is trivialised Crime is legitimisedAbortion is legalisedEverything is commercialised1,-0.999771
6,"RT @beverly_nkirote: @NAYAKenya @NMS_Kenya @MADACI_ke @lovemafrica @AYARHEP_KENYA @YouthActKE @rhnkorg @StraightTalkKe @ywli_info @CSA_Kenya @TICAH_KE @YourAuntyJane When @NMS_Kenya fails to prioritize the reproductive health of women and girls we then have increased cases of unsafe abortions , unintended pregnancies, increased cases of sexual and gender violence , increased cases of FGM and Child marriages #TheBudgetWeWant @MOH_Kenya",-0.999685
7,RT @MhakaTinatswe: The fact that you can serve 5 years in a Zimbabwean jail for having an abortion is disgusting and extremely nonsensical.,-0.999633
8,"@NAYAKenya @NMS_Kenya @MADACI_ke @lovemafrica @AYARHEP_KENYA @YouthActKE @rhnkorg @StraightTalkKe @ywli_info @CSA_Kenya @TICAH_KE @YourAuntyJane When @NMS_Kenya fails to prioritize the reproductive health of women and girls we then have increased cases of unsafe abortions , unintended pregnancies, increased cases of sexual and gender violence , increased cases of FGM and Child marriages #TheBudgetWeWant @MOH_Kenya",-0.999500
9,RT @corrasyne2: @NAYAKenya @NMS_Kenya @MADACI_ke @lovemafrica @AYARHEP_KENYA @YouthActKE @rhnkorg @StraightTalkKe @ywli_info @CSA_Kenya @TICAH_KE @YourAuntyJane When @NMS_Kenya fails to #BudgetForReproductiveHealth there's a rise in teenage pregnancies leading to the cases of unsafe abortion (which has terrible effects) and also there will be cases of higher spread of HIV@CSA_Kenya #TheBudgetWeWant #NAYAVoices,-0.999443


- Positive: film review (not related to the topic). Healthy nutrition (related to the topic). Gibraltar referendum to liberalise abortion bans. Positive messages for abortion information.


- Negative: Israel killings. Bad policies to give quality abortion services.

### Rape

In [40]:
rape = pd.read_sql_query('select count(num_retweets) as "count_rape" from srh where content regexp "rape";', engine)
rape

,count_rape
0,1782


In [117]:
rape_tw_pos = pd.read_sql_query('select distinct(content), sentiment from srh where content regexp "rape" order by sentiment desc limit 10;', engine)
rape_tw_pos

,content,sentiment
0,"The Mothers | Brit BennetIn honor of Mother's Day ðŸ˜… corny? I know but I enjoyed this book, very introspective on issues abortion; rape; feminism and a fun read! I loved Brit Bennett's writing style so we already know which book I am picking up next.. https://t.co/gUQpTkFTBf",0.962970
1,"""Coming together is a beginning, staying together is progress, and working together is success."" â€“ Henry FordHitting the Eastern part of Kenya with the most scores on teenage pregnancies, defilement, rapes and GBVs. Campaignes continues and thanks to Awino Foundation https://t.co/4YmXELbXcL",0.962158
2,"@TyJusthoe Hakuna dame hupata ball ki makosa, they just want to nag us here first things first amekupea akijua hayuko safe hujamrape, angemeza p2 ni option ama kama ako ready kupeana raw aende family planning before kupeana",0.816838
3,"RT @mainman_maina: @TyJusthoe Hakuna dame hupata ball ki makosa, they just want to nag us here first things first amekupea akijua hayuko safe hujamrape, angemeza p2 ni option ama kama ako ready kupeana raw aende family planning before kupeana",0.809672
4,"RT @SoulCity_SA: Bodily autonomy refers to a person's ability to make his or her own decisions concerning contraception, reproductive health care,&amp; sexual relationships. This includes the ability to say no to sex, yes to health care, &amp; the choice of when to have a family. #endrapecultureinafrica https://t.co/Nd7Ice9f8L",0.793391
5,"RT @YouAct_Europe: Check out our NEW publication ""#YouActPasstheMic! Time to give #underrepresentedSRHR topics the limelight!"" featuring comprehensive educational modules on #consent, #rapeculture, #decolonising, #disabilities and SRHR and #harmfulpractices!Download here: https://t.co/70hchFPVYb https://t.co/0xT8LVgy2U",0.656492
6,RT @cjwerleman: (NEW): The Uyghur Concentration Camp Experience.Mass arrest &gt; Interrogation &gt; Torture &gt; Forced Confession &gt; Forced Sterilization &gt; Forced Abortion/Adoption &gt; Gang Rape &gt; Forced Labor &gt; Forced Organ Harvesting &gt; Saudi-UAE Complicity!Watch here: ðŸ‘‰https://t.co/tnthDlVzv0,0.636958
7,"RT @Real_Theology: Just got the update that a girl who had been raped and got pregnant, who was considering abortion, chose life for her baby after counseling with the pro-life ministry that meets every week in front of PP.The sweet baby boy was born last week! 10lbs 6oz!!!!! Praise God!!!! ðŸ™ðŸ™Œ",0.636526
8,"RT @Safe_Abortion: . @UNFPA report: 50% of women in 57 developing countries denied bodily autonomy, subject to rape, forced sterilisation, virginity testing, FGM, reproductive abuse, healthcare &amp; contraceptive denial etc + decision making power &amp; education level linked: https://t.co/riMrLnoVWz âœŠ https://t.co/neUg9FrUum",0.611904
9,@YoungTrape1 You the type that walks around with 10 baby mamas and stis Calling,0.580088


In [118]:
rape_tw_neg = pd.read_sql_query('select distinct(content), sentiment from srh where content regexp "rape" order by sentiment limit 10;', engine)
rape_tw_neg

,content,sentiment
0,"RT @EuniceNyamuro: @Zamara_fdn @DorcasWege @Rosemar21055404 @MachukiMoraa @GlobalGirlsFund @SautiSkika @youth4SRHR @SRHRforAll @BitsSrhr @StraightTalkKe @girlsvoices1 @CHOICEforYouth @Polycomgirls1 @shesthefirst Young women and girls have engaged themselves in unprotected sex for money leading them to getting infections and viruses such as HIV /AIDs, others have died due to early pregnancies while young, raped #COVID19AndAGYW #zamaravoices",-0.998917
1,"RT @oremo_brian: @DeclaresGroupKe @PublicPathways @DSWKenya @VoiceandAgency1 @hivosrosa @tendasasa @voicetweetz @icrhk_official @YouthActKE @CanHCKenya @EndingFGM @equalitynow @AfyaJadili @dsw_de @StraightTalkKe Strict laws should be formulated and be implemented to curb issues relating to rape, FGM and embezzlement of funds meant to conduct sexual reproduction activities #InternationalWomensDay #IWD2021 #d",-0.998694
2,"RT @bennetowuonda: @Midrifthurinet @karuga @WillieOeba @KMapenai @ShankiAustine GBV includes:Sexual violence, including rape, sexual abuse, sexual exploitation and forced prostitutionDomestic violenceForced and early marriageHarmful traditional practices such as female genital mutilation, honour crimes, widow inheritanceTrafficking#MH4childrenandYouth",-0.998501
3,RT @Alawijamalileil: I'm kinda annoyed that 8 years since we've had women reps there has been:*no policies to take care of rape and fgm victims*no free sanitary pads in schools and universities* no national women empowerment programme*No GBV support center for womenAnd they earn fat paychecks,-0.998192
4,"@simiyu_m_wafula @LynetteNjeri5 GBV is forced recruitment of boys into fighting forces, It's 'corrective rape' of lesbians, It's refusing to register a transgendered person as an IDP because the sex on their documentation does not match their appearance, female genital mutilation.That's examples of GBV",-0.998019
5,"@DeclaresGroupKe @PublicPathways @DSWKenya @VoiceandAgency1 @hivosrosa @tendasasa @voicetweetz @icrhk_official @YouthActKE @CanHCKenya @EndingFGM @equalitynow @AfyaJadili @dsw_de @StraightTalkKe Strict laws should be formulated and be implemented to curb issues relating to rape, FGM and embezzlement of funds meant to conduct sexual reproduction activities #InternationalWomensDay #IWD2021 #d",-0.997611
6,"@AokoOtieno_ @kibeandy So sad how Society deems rape, spousal abuse, and genital mutilation as comedic topics, so long as the victims are males.",-0.997154
7,"RT @ItsChebetAbdi_: Children who are exposed to child labour are exposed to Sexual abuse, particularly sexual exploitation of girls by adults, rape, prostitution, early and unwanted pregnancy, abortion, Sexually Transmitted Diseases (STDs) and HIV/AIDS, drugs and alcoholism. #GhaiLarissaAward2021 https://t.co/zrPfTSkgPl",-0.997126
8,Rape is a serious crime!Murder is a serious crime too!Abortion is Murder!A crime doesn't heal the effects of another crime!#KenyaIsProLife https://t.co/1y46BppXUt,-0.996254
9,"RT @amwaafrika: ""I know that my fight on this African continent is a fight against patriarchy, poverty, sexism, homophobia, transphobia, ableism, FGM, rape, HIV/Aids, human and food insecurity, displacement, conflicts and the many atrocities we continue to face. https://t.co/ZLEpjvX3Vu",-0.996184


- Positive: Book review (not related to the topic). Campaign against rape, the autonomy of reproductive health care and freedom to choose. Chinese oppression against the Uyghur people (topic related, but not geographically).


- Negative: Condemn forced women to have unprotected sex and potential infections. The demand for better policies against rape and violent acts,  forced prostitution and FGM.

### Justice

In [42]:
justice = pd.read_sql_query('select count(num_retweets) as "count_justice" from srh where content regexp "justice";', engine)
justice

,count_justice
0,1852


In [119]:
justice_tw_pos = pd.read_sql_query('select distinct(content), sentiment from srh where content regexp "justice" order by sentiment desc limit 10;', engine)
justice_tw_pos

,content,sentiment
0,RT @IpasOrg: Now more than ever we're feeling the love for abortion providers â¤ï¸ðŸ’™ ðŸ’œ ðŸ’› ðŸ’š ðŸ§¡ Healthcare workers have gone above and beyond for us and we are so grateful for all that they do! #SRHR #reprorights #reprojustice https://t.co/oz977L7YkJ,0.995061
1,"RT @Jojo_Jazzmeen: Health, Sexual and Reproductive health rights, Women and Girls Education, climate justice, humanitarian support are all important to achieve global prosperity and wellbeing we should be investing more not CUT - stand on the right side of #Herstory #RaiseYourHand @GPforEducation",0.990657
2,RT @dsw_intl: How are sexual and reproductive health and rights (SRHR) ðŸ”— to #ClimateChange? Our latest ðŸ“½ï¸ highlights how investing in SRHR is essential to empowering women and girls to lead the response to the climate crisis ðŸ“¢ and to achieving climate justice. #GenerationEquality https://t.co/rp1WbUrh4o,0.989693
3,"RT @catherinenyamb1: Super stoked to see girls &amp; young women showing up at sessions at the #GenerationEquality Mexico Forum with strength, power and brilliance to articulate their priorities on #SRHR, #GBV, #Technology,#ClimateJustice, #FeministLeadership and #EconomicJustice #WhatGirlsWant https://t.co/dVimCeKkvd",0.983199
4,"RT @KibetBenard_: Michael Maina is a final year medical student at the University of Nairobi who is a passionate sexual and reproductive health rights, and justice (SRHRJ) advocate. #JumpAtTheSunLive https://t.co/pOGPa6dow4",0.978878
5,"RT @ReproRights: ðŸš¨We are headed back to the U.S. Supreme Court to defend the right to abortion established by #RoevWade.The fundamental right to make decisions about our bodies, lives, and futures is ESSENTIAL to the pursuit of racial, reproductive and economic justice.https://t.co/SHLIaaZ4KR",0.978655
6,"RT @IpasOrg: Thank you @DrTedros for taking time to talk about #SRHR issues. We agree, we can only address this issue through #healthequity, partnership &amp; strengthening health systems. #reprojustice #reprohealth https://t.co/juV3T6N3RR",0.975999
7,"RT @IpasOrg: Today, at the #GenerationEqualityForum, Ipas is proud to share our partnership with @WomenDeliver @WEDO_worldwide &amp; more orgs to commit to collective action and joint advocacy at the nexus of #GenderEquality, #SRHR and #ClimateJustice. https://t.co/irawcvbGPH",0.975749
8,"Michael Maina is a final year medical student at the University of Nairobi who is a passionate sexual and reproductive health rights, and justice (SRHRJ) advocate. #JumpAtTheSunLive https://t.co/pOGPa6dow4",0.964377
9,"We at Adolescents Health Forum committed to working with partners who share and strengthen our commitment to secure long-term gains in HIV, sexual and reproductive health and rights, and equitable access to health with a social justice lens @MalecheAllan @NACC_Kenya @NEPHAKKENYA https://t.co/nMGOHVlnMH",0.963280


In [120]:
justice_tw_neg = pd.read_sql_query('select distinct(content), sentiment from srh where content regexp "justice" order by sentiment limit 10;', engine)
justice_tw_neg

,content,sentiment
0,"RT @KamauD33: The fight against Rape, FGM, domestic violence in Kenya needs the input of Kenyan men. Men should stand up and be protectors even as they provide. #JusticeforVelvine #HumanRights #rape",-0.995616
1,Sharon Otieno was brutally raped and murdered with a foetus in her womb 3 years ago. Why's there still a court case is the only question because justice delayed is justice denied. Seems like jokes.,-0.994828
2,"RT @fahe_k: @NAYAKenya @NMS_Kenya @MADACI_ke @lovemafrica @AYARHEP_KENYA @YouthActKE @rhnkorg @StraightTalkKe @ywli_info @CSA_Kenya @TICAH_KE @YourAuntyJane When @NMS_Kenya fails to prioritize Womxn and Girls in all their Diversities, this leads to alot of SRH injustices such as Stigma and Discrimination leading to those in need not seeking for services that they are rightfully entitled to. #TheBudgetWeWant",-0.994136
3,RT @FrankoWasubu: Shuttered ambitions!Killed dreams.Soul lost.Parents in grief and agony.Killed foetus.The cry is so loud.#JusticeForSharonOtieno https://t.co/uBEhZgUAku,-0.993647
4,"@NAYAKenya @NMS_Kenya @MADACI_ke @lovemafrica @AYARHEP_KENYA @YouthActKE @rhnkorg @StraightTalkKe @ywli_info @CSA_Kenya @TICAH_KE @YourAuntyJane When @NMS_Kenya fails to prioritize Womxn and Girls in all their Diversities, this leads to alot of SRH injustices such as Stigma and Discrimination leading to those in need not seeking for services that they are rightfully entitled to. #TheBudgetWeWant",-0.992494
5,Shuttered ambitions!Killed dreams.Soul lost.Parents in grief and agony.Killed foetus.The cry is so loud.#JusticeForSharonOtieno https://t.co/uBEhZgUAku,-0.991598
6,@17johnmuniu What option? Abortion is inexcusable bro. You can never justify this inhumane act. It is an injustice to the unborn#NoToAbortionKE,-0.991409
7,RT @RahelBerheS: #TigrayWarRapeVictims: In Mekelle many women are asking for a post-pill as a precaution for avoiding unwanted pregnancy due to an imminent danger of being raped by the Ethiopian and Eritrean forces. Justice for the #Tigrayan women &amp; girls. @AnnLinde https://t.co/ZDmzbjBHUG,-0.990776
8,RT @LisaMaryAnne03: @NAYAKenya @loveablekathia @KombeMartha @EssyAimee @YourAuntyJane @YouthActKE @lovemafrica @rhnkorg @StraightTalkKe @KenyaSRHR @CSA_Kenya @Varaq @victorrasugu @tdhnl_africa Equal access to quality education for AGYW means ending or reducing injustices against them such as early marriages and FGM because they will be in learning institutions and not in endangered home environments. #SheLeadsKenya #SheLeadsYouth #IYD2021 #NAYAVoices,-0.986304
9,"RT @saoyogriffith: To the women who lost lives at childbirth,those forcefully sterilized, those stigmatized for their HIV status,those who are trafficked, those who die from unsafe abortion, those killed for not conforming, I may not reverse the injustices but I try ensure it NEVER happens againðŸ™ https://t.co/DfUvDEt4Of",-0.985037


- Positive: Promotion of reproduction rights and justice. Reproduction justice related to Climate Change.


- Negative: Demand for men to tackle the domestic violence problem in Kenya. Justice denied or delayed. SRH injustice and discrimination.

### Freedom

In [43]:
freedom = pd.read_sql_query('select count(num_retweets) as "count_freedom" from srh where content regexp "freedom";', engine)
freedom

,count_freedom
0,1310


In [112]:
freedom_tw_pos = pd.read_sql_query('select distinct(content), sentiment from srh where content regexp "freedom" order by sentiment desc limit 10;', engine)
freedom_tw_pos

,content,sentiment
0,"RT @Eve_odhis: Happy 2ï¸âƒ£0ï¸âƒ£2ï¸âƒ£1ï¸âƒ£ to you all â™¥ï¸â™¥ï¸ðŸŽˆ#Welcome2021 ðŸŽˆðŸŽ‰ and be great to our work as advocates, activists, champions, leaders and feminists fighting to ensure an equal and balanced society where we all enjoy our reproductive rights and freedom fully.#HappyNewYear2021 https://t.co/rvhbkgQcWa https://t.co/4NgHOR0bXJ",0.994899
1,@power_dada ... Thank you so much and I am also super proud of the work that you do for Women and girls in Kisumu. To more of reproductive health freedom. Let's keep it #BongaNaAuntyJane ðŸ’•,0.993405
2,"RT @adventurefrom: New Blog Post Alert ðŸ“¢ðŸ“¢ðŸ“¢ ""I encourage you this Pride Month and beyond to honour the true spirit of pride by resisting the suppression of African sexual rights and taking a bold unwavering stand for sexual freedom and pleasure."" @JustDerby #Pridehttps://t.co/6MdihEMJoM https://t.co/CoXGFGbo5O",0.992034
3,"RT @SAAFfund: Today, on #InternationalWomensDay we thank and celebrate all the amazing SAAF grantee partners who work to improve reproductive rights and freedoms around the world ðŸŒŽ #IWD2021 #IWD21 https://t.co/n1KD6hYKHb",0.990214
4,"RT @scheafferoo: To the wonderful women who continue to hold, support, teach and guide necessary conversations on contraceptive freedom, THANK YOU! @Zindzi_K ðŸŒŸ@rachaelmwiksðŸŒŸ @QueenGathoni ðŸŒŸ@YvonneOkwaraðŸŒŸ @wanjaah ðŸŒŸ @abatoni ðŸŒŸðŸŒ»ðŸ’›ðŸ™ðŸ¾",0.986586
5,"RT @ShareNetNL: Amazing opportunity alert!Are you a passionate SRHR youth advocate? Then check out this vacancy for the Youth Ambassador for SRHR, Gender Equality and Freedom of Choice (JASRGR)https://t.co/vGhBShiIQ9- Knowledge of Dutch language is required",0.985538
6,RT @KMapenai: Access to comprehensive sexual reproductive health information creates an enabling environment for women to lead full productive lives and have the freedom to make healthcare choices that are best for them. #Act2EndFGMÂ @AkiliDada @TuwezesheDada @FORWARDUK#sisterhood#Act2EndFGM,0.985072
7,"RT @Komuono: #WorldPressFreedomDayKE Media has been a platform for SRHR voices. On this day, we celebrate every brave soul out there who has been part of the journey.@KELINKenya https://t.co/DfXbeTpOVd",0.984126
8,"RT @KELINKenya: TO realize SRHR, we all need to be informed about and have access to a safe, effective, affordable and acceptable contraception method of their choice #WorldPressFreedomDayKE",0.980702
9,RT @Komuono: #WorldPressFreedomDayKE is such a special day and we in SRHR want to thank YOU the journalists who are in the frontline in helping spread the voices of SRHR.@KELINKenya,0.978997


In [121]:
freedom_tw_neg = pd.read_sql_query('select distinct(content), sentiment from srh where content regexp "freedom" order by sentiment limit 10;', engine)
freedom_tw_neg

,content,sentiment
0,"RT @Mukami_Mungai: @KegediL @Alvinmwangi254 @firdausamo @G_MachelTrust @UNFPAKen @equalitynow @UNFPA_ESARO @YouthAmbSRHR @SRHRAfricaTrust @SRHRforAll @MoH_DRMH @MOH_Kenya @DrMercyHealth @DrMercyKorir @JulittaOnabanjo @IYAFPKenya @ConnectAID_int @KenyaSRHR @FemnetProg @crawntrustkenya @feminstyle -Address gender inequalities, power imbalances and male dominance in order for women to meaningfully claim and enjoy freedom from violence, abuse, coercion and discrimination. #MaputoProtocol #GlobalGoal",-0.999781
1,"@KegediL @Alvinmwangi254 @firdausamo @G_MachelTrust @UNFPAKen @equalitynow @UNFPA_ESARO @YouthAmbSRHR @SRHRAfricaTrust @SRHRforAll @MoH_DRMH @MOH_Kenya @DrMercyHealth @DrMercyKorir @JulittaOnabanjo @IYAFPKenya @ConnectAID_int @KenyaSRHR @FemnetProg @crawntrustkenya @feminstyle -Address gender inequalities, power imbalances and male dominance in order for women to meaningfully claim and enjoy freedom from violence, abuse, coercion and discrimination. #MaputoProtocol #GlobalGoals #SRHR #ActForEqua",-0.999563
2,"Girls and women with disabilities have the right to freedom from sexual abuse and violence, including unnecessary and unwarranted abortion.#DisabiltyRightsKe",-0.991166
3,"@RealistKindness @BravinYuri Adults are messed up because they don't teach their own children sex education, they lock them up, and don't give them freedom so the government is forced to think for them since not everyone can think for themselves..",-0.982154
4,"RT @RitahAnindo: One death caused a preventable cause is way to much, when the media covers one death caused by unsafe abortion, it represents thousands of similar cases #WorldPressFreedomDayKE https://t.co/9x8c9IjGio",-0.971131
5,RT @RitahAnindo: Press freedom means speaking about the realities and experience of young womenThe struggle for contraceptionThe struggle for safe abortion servicesThe struggle for informationThe violence cases and struggle to get justiceThe preventable deaths#WorldPressFreedomDayKE https://t.co/8nXblxVLkQ,-0.966063
6,"@angelamavalla Like the pro abortion females crowd like to yell out, my body my choice... My freedom should not end where your fears begin.",-0.963233
7,"One death caused a preventable cause is way to much, when the media covers one death caused by unsafe abortion, it represents thousands of similar cases #WorldPressFreedomDayKE https://t.co/9x8c9IjGio",-0.957720
8,"@NeutronLiam @CameronJoseph76 @RosaRubicon Yeah, and as such priests have the freedom to defile young boys and say that condoms are worse than HIV",-0.957251
9,"""#Abortionstigma violates the #rights of women and girls to access health, psychosocial support, and freedom from cruel, inhumane, and degrading treatment."" #SRHRisessential #SRHR #Reproductionhealthrights #mondaythoughts https://t.co/HL7mGEQGJs",-0.953136


- Positive: Celebration of people's work for ensuring equality and freedom for the society. Proud of the work done by the women. Spirit of pride and resisting suppression of African sexual rights. International Women's Day celebration.


- Negative: Denounce power imbalance, male dominance, lack of female freedom and discrimination, the right of freedom for the female population with disabilities. More access to health, psychosocial support for women. Demand of women rights. 

### Human right(s)

In [44]:
human_right = pd.read_sql_query('select count(num_retweets) as "count_hum_right" from srh where content regexp "human right";', engine)
human_right

,count_hum_right
0,3196


In [122]:
h_right_tw_pos = pd.read_sql_query('select distinct(content), sentiment from srh where content regexp "human right" order by sentiment desc limit 10;', engine)
h_right_tw_pos

,content,sentiment
0,RT @Africa_Oxford: We are proud to support @OxHRH #ShapingtheFuture series that highlights the role of human rights law to achieve sexual and reproductive health for women around the world. Watch the first episode here: https://t.co/VaO8CQS591 https://t.co/ChdI9RZo1z,0.983926
1,"RT @kala_afrika: Today in celebrating women inspiring Change in our society,I greet you in the name of Dr. Stella Nyanzi @nyanzi_dr .Stella Nyanzi is a Ugandan human rights activist, poet, medical anthropologist, feminist, queer rights activist, and scholar of sexuality including SRHR. https://t.co/2H16xnKlF0",0.982111
2,"RT @DollarmanKE: Comprehensive Sexuality Education supports young people's empowerment by improving their analytical, communication, and other life skills for health and well-being in relation to sexuality, human rights, and values.@TICAH_KE @NCPD_Kenya @EduMinKenya #IamForCSEKE https://t.co/3QZKpUjOmt",0.977990
3,RT @IpasOrg: #Abortion is:âœ”ï¸ Safeâœ”ï¸ Effectiveâœ”ï¸ Essentialâœ”ï¸ A human right https://t.co/8QU9vjdECV,0.975319
4,"Comprehensive Sexuality Education supports young people's empowerment by improving their analytical, communication, and other life skills for health and well-being in relation to sexuality, human rights, and values.@TICAH_KE @NCPD_Kenya @EduMinKenya #IamForCSEKE https://t.co/3QZKpUjOmt",0.972112
5,"1 in every 4 married women in Kenya has unmet need for family planning.Maintaining good #SRH means people can access accurate information and safe, effective &amp; affordable contraceptive methods of their choice.Access to contraceptives is a human right, that should be prioritized! https://t.co/3EvQsAtXdg",0.969528
6,"RT @amnestyusa: ðŸ—£ï¸ðŸ“£Women's rights are human rights.Sexual and reproductive rights are human rights.Thanks for saying it loud and proud, @SecBlinken! ðŸŽ‰ðŸŽ‰ðŸŽ‰",0.968287
7,RT @ReproRights: We applaud the work of @fred_matic on the #SRHR report and welcome the adoption of the report by @EP_GenderEqual. ðŸ‘The report affirms that #SRHR are fundamental human rights and we hope the EP plenary will endorse it. https://t.co/IOpX49V3MV,0.966150
8,"RT @maita_shelmith: Sexual reproductive Rights are Human Rights .Every woman has a right to the highest attainable standard of Health ,to safe reproductive choices and to high quality health care including right to safe abortion services .#teenpregnancyke #SafeAbortionKE https://t.co/pYKG4iaKsC",0.965769
9,"Sexual reproductive Rights are Human Rights .Every woman has a right to the highest attainable standard of Health ,to safe reproductive choices and to high quality health care including right to safe abortion services .#teenpregnancyke #SafeAbortionKE https://t.co/pYKG4iaKsC",0.965769


In [123]:
h_right_tw_neg = pd.read_sql_query('select distinct(content), sentiment from srh where content regexp "human right" order by sentiment limit 10;', engine)
h_right_tw_neg

,content,sentiment
0,"RT @IamMithamo: @DanielMaithyaKE @soydasomalia @Ypeer_Puntland @YPEER_Somalia @Anders_Thomsen @GPtoEndFGM @GPtoEndViolence @voicesofyouth FGM is a harmful practice and is unacceptable from a human rights as well as a public health perspective, regardless of who performs it.#SOYDAsomalia https://t.co/svSARtLP9Q",-0.995986
1,"@DanielMaithyaKE @soydasomalia @Ypeer_Puntland @YPEER_Somalia @Anders_Thomsen @GPtoEndFGM @GPtoEndViolence @voicesofyouth FGM is a harmful practice and is unacceptable from a human rights as well as a public health perspective, regardless of who performs it.#SOYDAsomalia https://t.co/svSARtLP9Q",-0.993294
2,"FGM is a harmful practice and is unacceptable from a human rights as well as a public health perspective, regardless of who performs it.@unicefsomalia @shf_somalia@UNHCRSom @UKinSomalia@UNSomalia#SOYDAsomalia https://t.co/9EYQohdzpr",-0.991932
3,"RT @kenvicglobal: @SirMianoJ @caren_cheroe @MenEndFGM @ActionAid_Kenya @UNFPAKen @AFGMBoard @EliasMuindi @PCKemei @K_thiakunu FGM is completely a violation on human rights ,an inhuman and degrading treatment and right to life when procedure results in death",-0.988714
4,"RT @AFGMBoard: While the county is not one of the hot spot counties where the FGM practice is rife, we maintain FGM is the worst form of human rights violations meted on women &amp; take any threats of the perpetuation of the vice seriously as the only acceptable number is ZERO. #EndFGM https://t.co/kpM90sYOE2",-0.985719
5,RT @alicebroster1: There's a target to end FGM by 2030 but as schools and clinics closed it's left some of the most vulnerable people in a much worse off positionFGM is abusive and needless- a total violation of human rights https://t.co/ouJ8ve5BBG,-0.984662
6,"@Zamara_fdn @kefeado @NetworkAthena @forequality_mw @GirlsWomenPower @YawiKenya @YEMKenya @CSA_Kenya @KenyaSRHR @KELINKenya @TICAH_KE @covaw @wgnrr_africa @amwaafrika @GBVnet Legal environments need to prioritise laws, policies and programmes that address the human rights and gender-related factors that cause a heightened risk violence greatly contributing to a rise in teenage pregnancies, HIV new infections among others #CSW65Ke #ZamaraVoices",-0.982699
7,RT @CanadaFgm: Dear ðŸ‡¨ðŸ‡¦ PM â¦â¦@JustinTrudeauâ©Every 10 seconds a girl is forced through Female Genital Mutilation4.1 million girls a yearWe ask our citizens to compare these 2 tweets &amp; judge how serious your 'feminist' government is about standing up for girls' human rights. https://t.co/qIpoVwsoXV,-0.979773
8,"RT @loveablekathia: ""SRHR problems are entirely preventable thus investing in fulfilling the human rights of women &amp; girls,as gender discrimination is one of the leading determinants of poor health and unwanted SRHR outcomes as well as key to address inequities""#RaisingOurVoices #NAYAVoices https://t.co/EvbA9K6eEv",-0.977738
9,"Forcing women and girls to continue a pregnancy or turn to unsafe abortion when their life is in danger, or when a pregnancy is unviable or caused by rape or incest, is dangerous and cruel and violates human rights https://t.co/tCetye1iUf",-0.976087


- Positive: Support for human rights. Celebration and greetings for people whose fighting for women's rights. Sexual education and young people's empowerment. The demand for prioritisation of contraceptive access. Reclaim of sexual and reproductive rights. 


- Negative: Mainly demand the end of violence. Condemn FGM as a harmful practice and human rights violation.

### Health

In [45]:
health = pd.read_sql_query('select count(num_retweets) as "count_health" from srh where content regexp "health";', engine)
health

,count_health
0,46103


In [124]:
health_tw_pos = pd.read_sql_query('select distinct(content), sentiment from srh where content regexp "health" order by sentiment desc limit 10;', engine)
health_tw_pos

,content,sentiment
0,"RT @Consumers_Kenya: @siayifabian The vision of achieving a safe, accurate, high quality, free, acceptable and voluntary family planning services and information will further aid in empowering and encouraging our women and young girls to lead better, healthier, and happier lives. #TujulishaneFamilyPlanning",0.999201
1,FertipilPlus for Menâž¡ï¸ Boosts and improves sperm countâž¡ï¸ Improves sperm mobility and morphologyâž¡ï¸ Improves spermatogenesisâž¡ï¸ Boosts natural reproductive health and sexual functionTel. 0794 429 991https://t.co/HDrmuJkabp#sperm #spermcount #fertipil #fertipilplusformen https://t.co/4WX5s78Cy9,0.998780
2,FertipilPlus for Menâž¡ï¸ Boosts and improves sperm countâž¡ï¸ Improves sperm mobility and morphologyâž¡ï¸ Improves spermatogenesisâž¡ï¸ Boosts natural reproductive health and sexual functionTel. 0794 429 991https://t.co/IBJbWzU7I8#sperm #spermcount #fertipil #fertipilplusformen https://t.co/02JqTZw8dU,0.998780
3,FertipilPlus for Menâž¡ï¸ Boosts and improves sperm countâž¡ï¸ Improves sperm mobility and morphologyâž¡ï¸ Improves spermatogenesisâž¡ï¸ Boosts natural reproductive health and sexual functionTel. 0794 429 991https://t.co/Vj6VRDPIoj#sperm #spermcount #fertipil #fertipilplusformen https://t.co/2RZwR485Pw,0.998780
4,FertipilPlus for Menâž¡ï¸ Boosts and improves sperm countâž¡ï¸ Improves sperm mobility and morphologyâž¡ï¸ Improves spermatogenesisâž¡ï¸ Boosts natural reproductive health and sexual functionTel. 0794 429 991https://t.co/Vj6VRDPIoj#sperm #spermcount #fertipil #fertipilplusformen https://t.co/GJ0jgHfPXr,0.998780
5,"RT @AmrefICD: ICD's Knowledge SUCCESS project excited to announce the launch of FP insight, a new platform built by and for family planning and reproductive health (FP/RH) professionals to discover and curate resources relevant to your work. #FPInsight https://t.co/uTMT24w6Jq https://t.co/kmqYUW7ByO",0.998729
6,"@siayifabian The vision of achieving a safe, accurate, high quality, free, acceptable and voluntary family planning services and information will further aid in empowering and encouraging our women and young girls to lead better, healthier, and happier lives. #TujulishaneFamilyPlanning",0.998318
7,"ICD's Knowledge SUCCESS project excited to announce the launch of FP insight, a new platform built by and for family planning and reproductive health (FP/RH) professionals to discover and curate resources relevant to your work. #FPInsight https://t.co/uTMT24w6Jq https://t.co/kmqYUW7ByO",0.997718
8,"FertipilPlus for Menâž¡ï¸ Boosts and improves sperm countâž¡ï¸ Improves sperm mobility and morphologyâž¡ï¸ Improves spermatogenesisâž¡ï¸ Boosts natural reproductive health and sexual functionKshs 2,495/- Tel. 0794 429 991 #sperm #spermcount #fertipil #fertipilplusformen #pregnant https://t.co/ULFxePtyrm",0.997644
9,"FertipilPlus for Menâž¡ï¸ Boosts and improves sperm countâž¡ï¸ Improves sperm mobility and morphologyâž¡ï¸ Improves spermatogenesisâž¡ï¸ Boosts natural reproductive health and sexual functionKshs 2,495/- Tel. 0794 429 991#sperm #spermcount #fertipil #fertipilplusformen #pregnant https://t.co/4zsI5IVTMx",0.997644


In [125]:
health_tw_neg = pd.read_sql_query('select distinct(content), sentiment from srh where content regexp "health" order by sentiment limit 10;', engine)
health_tw_neg

,content,sentiment
0,"RT @maita_shelmith: @NAYAKenya @NMS_Kenya @MADACI_ke @lovemafrica @AYARHEP_KENYA @YouthActKE @rhnkorg @StraightTalkKe @ywli_info @CSA_Kenya @TICAH_KE @YourAuntyJane When @NMS_Kenya fails to prioritize SRHR and budgetary allocations for SRH ,it encourages the continuation of quack abortion services,increases the alarming numbers of unwanted teen pregnancies ,unsafe abortions and even death .#TheBudgetWeWant #BudgetForReproductiveHealth",-0.999942
1,"@NAYAKenya @NMS_Kenya @MADACI_ke @lovemafrica @AYARHEP_KENYA @YouthActKE @rhnkorg @StraightTalkKe @ywli_info @CSA_Kenya @TICAH_KE @YourAuntyJane When @NMS_Kenya fails to prioritize SRHR and budgetary allocations for SRH ,it encourages the continuation of quack abortion services,increases the alarming numbers of unwanted teen pregnancies ,unsafe abortions and even death .#TheBudgetWeWant #BudgetForReproductiveHealth",-0.999906
2,"RT @MADACI_ke: @NAYAKenya @NMS_Kenya @lovemafrica @AYARHEP_KENYA @YouthActKE @rhnkorg @StraightTalkKe @ywli_info @CSA_Kenya @TICAH_KE @YourAuntyJane Q4:When NMS fails to #BudgetForReproductiveHealth women and girls will be denied access to Sexual and Reproductive Health services and information hence putting the at risk of teenage pregnancies, unsafe abortions and STIs and HIV infections.#TheBudgetWeWant",-0.999846
3,"@NAYAKenya @NMS_Kenya @lovemafrica @AYARHEP_KENYA @YouthActKE @rhnkorg @StraightTalkKe @ywli_info @CSA_Kenya @TICAH_KE @YourAuntyJane Q4:When NMS fails to #BudgetForReproductiveHealth women and girls will be denied access to Sexual and Reproductive Health services and information hence putting the at risk of teenage pregnancies, unsafe abortions and STIs and HIV infections.#TheBudgetWeWant",-0.999790
4,"RT @Mukami_Mungai: @KegediL @Alvinmwangi254 @firdausamo @G_MachelTrust @UNFPAKen @equalitynow @UNFPA_ESARO @YouthAmbSRHR @SRHRAfricaTrust @SRHRforAll @MoH_DRMH @MOH_Kenya @DrMercyHealth @DrMercyKorir @JulittaOnabanjo @IYAFPKenya @ConnectAID_int @KenyaSRHR @FemnetProg @crawntrustkenya @feminstyle -Address gender inequalities, power imbalances and male dominance in order for women to meaningfully claim and enjoy freedom from violence, abuse, coercion and discrimination. #MaputoProtocol #GlobalGoal",-0.999781
5,"RT @beverly_nkirote: @NAYAKenya @NMS_Kenya @MADACI_ke @lovemafrica @AYARHEP_KENYA @YouthActKE @rhnkorg @StraightTalkKe @ywli_info @CSA_Kenya @TICAH_KE @YourAuntyJane When @NMS_Kenya fails to prioritize the reproductive health of women and girls we then have increased cases of unsafe abortions , unintended pregnancies, increased cases of sexual and gender violence , increased cases of FGM and Child marriages #TheBudgetWeWant @MOH_Kenya",-0.999685
6,RT @ThePeoplesHubUK: @ThePeoplesHubUN @BanishFGM @GoogleExpertUK @healthertainer @PankhurstCentre @AndyBurnhamGM @DNW2018 @behindbras @evefrancisholt @elsazekeng @CMBoggiano @NorthPowerWomen @univinclusion @susybotello @dbgrinberg @WomenOccupyHwd @AdaTheShow @FolukeMichael @LisaRingGA01 @amyklobuchar @MarshaCollier @charlotteh71 @WendyESlater @HiboWardere @blueskyssun @InspiringSTEM_N @GoodnessCIC @MrsEdithRigby @AdamRogers2030 @SDGsTunisia @Chiefson08 @SoyCibelino @Dream_Rise_F @iBuseGedik @gye,-0.999591
7,"@KegediL @Alvinmwangi254 @firdausamo @G_MachelTrust @UNFPAKen @equalitynow @UNFPA_ESARO @YouthAmbSRHR @SRHRAfricaTrust @SRHRforAll @MoH_DRMH @MOH_Kenya @DrMercyHealth @DrMercyKorir @JulittaOnabanjo @IYAFPKenya @ConnectAID_int @KenyaSRHR @FemnetProg @crawntrustkenya @feminstyle -Address gender inequalities, power imbalances and male dominance in order for women to meaningfully claim and enjoy freedom from violence, abuse, coercion and discrimination. #MaputoProtocol #GlobalGoals #SRHR #ActForEqua",-0.999563
8,"@NAYAKenya @NMS_Kenya @MADACI_ke @lovemafrica @AYARHEP_KENYA @YouthActKE @rhnkorg @StraightTalkKe @ywli_info @CSA_Kenya @TICAH_KE @YourAuntyJane When @NMS_Kenya fails to prioritize the reproductive health of women and girls we then have increased cases of unsafe abor

- Positive: Demand for safe, free and quality family planning. Advertisement from a fertility clinic and a family planning platform.


- Negative: The failure of budgetary allocations for abortion services. The increase of sexual and gender violence.

### Choice

In [47]:
choice = pd.read_sql_query('select count(num_retweets) as "count_choice" from srh where content regexp "choice";', engine)
choice

,count_choice
0,6568


In [126]:
choice_tw_pos = pd.read_sql_query('select distinct(content), sentiment from srh where content regexp "choice" order by sentiment desc limit 10;', engine)
choice_tw_pos

,content,sentiment
0,"RT @zahnooraH: Our national and county gvnt must ensure comprehensive sexuality education is accessible for all Adolescents and young ple to gain knowledge and develope the necessary skills to make conscious, healthy and respectful choices abt sexuality and relationships #BongaNaOne2one",0.997572
1,"RT @kilifi_youth: Our National and County government must ensure that comprehensive sexuality education is accessible for all AYP to gain knowledge and develop the necessary skills to make conscious, healthy, and respectful choices about relationships and sexuality.#GetInformed #BongaNaOne2one",0.997565
2,"RT @ngila_emmanuel: Our National/ County government must ensure that comprehensive sexuality education is accessible for all adolescents, and young people, to gain knowledge and develop the necessary skills to make conscious, healthy, respectful choices about relationships and sexuality #GetInformed",0.997067
3,"RT @leujonson: Our governments must ensure that comprehensive sexuality education is accessible for all adolescents, and young people, to gain knowledge and develop the necessary skills to make conscious, healthy, and respectful choices about relationships and sexuality.#GetInformed",0.996658
4,"Our National/ County government must ensure that comprehensive sexuality education is accessible for all adolescents, and young people, to gain knowledge and develop the necessary skills to make conscious, healthy, respectful choices about relationships and sexuality #GetInformed",0.995617
5,"Our National and County government must ensure that comprehensive sexuality education is accessible for all AYP to gain knowledge and develop the necessary skills to make conscious, healthy, and respectful choices about relationships and sexuality.#GetInformed #BongaNaOne2one",0.995526
6,"RT @bettymtuweta: National and County government must ensure comprehensive sexuality education is accessible for all adolescents and young people to gain knowledge and develop the necessary skills to make conscious, healthy, and respectful choices about relationships and sexuality #GetInformed",0.995510
7,"RT @Melody00381708: National and County government must ensure comprehensive sexuality education is accessible for all adolescents and young people to gain knowledge and develop the necessary skills to make conscious, healthy, and respectful choices about relationships and sexuality. #GetInformed",0.995056
8,"Our governments must ensure that comprehensive sexuality education is accessible for all adolescents, and young people, to gain knowledge and develop the necessary skills to make conscious, healthy, and respectful choices about relationships and sexuality.#GetInformed",0.994230
9,"National and County government must ensure comprehensive sexuality education is accessible for all adolescents and young people to gain knowledge and develop the necessary skills to make conscious, healthy, and respectful choices about relationships and sexuality #GetInformed",0.992378


In [127]:
choice_tw_neg = pd.read_sql_query('select distinct(content), sentiment from srh where content regexp "choice" order by sentiment limit 10;', engine)
choice_tw_neg

,content,sentiment
0,"RT @Brace_for_Impat: @General_Oluchi This thing is so painful, it's painful bcus a man is paying for a choice he didn't make, what's worse if a man wanted the child, nd the woman objects she can abort it nd nothing will happen, so this isn't about ""a human being involved"" SO UNFAIR",-0.999273
1,"RT @EuniceNyamuro: @Zamara_fdn @DorcasWege @Rosemar21055404 @MachukiMoraa @GlobalGirlsFund @SautiSkika @youth4SRHR @SRHRforAll @BitsSrhr @StraightTalkKe @girlsvoices1 @CHOICEforYouth @Polycomgirls1 @shesthefirst Young women and girls have engaged themselves in unprotected sex for money leading them to getting infections and viruses such as HIV /AIDs, others have died due to early pregnancies while young, raped #COVID19AndAGYW #zamaravoices",-0.998917
2,RT @Valenti26058615: @Zamara_fdn @DorcasWege @Rosemar21055404 @MachukiMoraa @GlobalGirlsFund @SautiSkika @youth4SRHR @SRHRforAll @BitsSrhr @StraightTalkKe @girlsvoices1 @CHOICEforYouth @Polycomgirls1 @shesthefirst With corona and the lockdowns that have been put in place a lot of young women have been unable to access contraceptives therefore affected them greatly others even getting unwanted pregnancies #COVID19ANDAGYW #ZamaraVoices,-0.997694
3,RT @Tomma_kiteto: @stagemediaarts @Real_Nixon_ @J_Nyando @YourAuntyJane @BungomaCountyKe @CWangamati_ @safe2choose @BYCke_ @YouthActKE @CboHype @MSIchoices @DreamAchieversk @BungomaRmncah @DeclaresKenya @AYGKenya @reprodrive This the gobal obligation to empower girls and women to access services on prevention treatment and care without travelling a long distance or wasting time#SRHR,-0.997434
4,RT @MachukiMoraa: @Zamara_fdn @DorcasWege @Rosemar21055404 @GlobalGirlsFund @SautiSkika @youth4SRHR @SRHRforAll @BitsSrhr @StraightTalkKe @girlsvoices1 @CHOICEforYouth @Polycomgirls1 @shesthefirst With lockdown leading to a shadow of pandemic of GBV and rates of child marriage and FGM increase rapidly#COVID19andAGYW#zamaravoices@MachukiMoraa@Estheraoko@DorcasWege,-0.996523
5,"RT @JumaKerryle: @Zamara_fdn @DorcasWege @Rosemar21055404 @MachukiMoraa @GlobalGirlsFund @SautiSkika @youth4SRHR @SRHRforAll @BitsSrhr @StraightTalkKe @girlsvoices1 @CHOICEforYouth @Polycomgirls1 @shesthefirst There has been increase in GBV and child marriages,teenage pregnancy and FGM in adolescent girls and young women's SRHR#Covid_19 andAGYW#zamaraVoices",-0.995583
6,"RT @loveablekathia: @Zamara_fdn @DorcasWege @Rosemar21055404 @MachukiMoraa @GlobalGirlsFund @SautiSkika @youth4SRHR @SRHRforAll @BitsSrhr @StraightTalkKe @girlsvoices1 @CHOICEforYouth @Polycomgirls1 @shesthefirst Many women &amp; girls are not able to access the necessary SRH services during the crisis, such as emergency contraception &amp; safe abortion in a timely manner. #COVID19AndAGYW",-0.994402
7,"@Zamara_fdn @DorcasWege @Rosemar21055404 @MachukiMoraa @GlobalGirlsFund @SautiSkika @youth4SRHR @SRHRforAll @BitsSrhr @StraightTalkKe @girlsvoices1 @CHOICEforYouth @Polycomgirls1 @shesthefirst Many women &amp; girls are not able to access the necessary SRH services during the crisis, such as emergency contraception &amp; safe abortion in a timely manner. #COVID19AndAGYW",-0.993678
8,RT @fahe_k: @Zamara_fdn @DorcasWege @Rosemar21055404 @MachukiMoraa @GlobalGirlsFund @SautiSkika @youth4SRHR @SRHRforAll @BitsSrhr @StraightTalkKe @girlsvoices1 @CHOICEforYouth @Polycomgirls1 @shesthefirst Access to SRH services has been limited &amp; others like #LGBTQ people denied services leading to alot of stigma &amp; Discrimination.#COVID19AndAGYW#ZamaraVoices https://t.co/XCIgvpYSIM,-0.992843
9,"@ItsMutai FGM is harmful. It is like legalizing suicide. It is someone's choice to take away their life, but it's still illegal.",-0.992052


- Positive: The demand for better sexual education for adolescents to develop the necessary skills to make conscious choices to the Government (more funds).


- Negative: Indecision between the parents at the moment to have a child. The condemnation for forcing women to have sex for money. Inability to access contraceptives as a consequence of Covid lockdowns.

### Contraceptive

In [48]:
contraceptive = pd.read_sql_query('select count(num_retweets) as "count_contraceptive" from srh where content regexp "contraceptive";', engine)
contraceptive

,count_contraceptive
0,13310


In [129]:
contra_tw_pos = pd.read_sql_query('select distinct(content), sentiment from srh where content regexp "contraceptive" order by sentiment desc limit 10;', engine)
contra_tw_pos

,content,sentiment
0,RT @DKTchangeslives: DKT is excited to celebrate our #FP2030 commitment to improving the lives of women and girls worldwide through contraceptive access!Read more in our recent announcement here: https://t.co/NsVY07YJXA,0.997231
1,"Celebrate love this valentine without the fear of unplanned pregnancy, Weka contraceptive mpangoni, Marie Stopes Kenya is offering free contraceptive at their Health Clinic near you, Whatsapp them on 0709819001 for more information. ~#contraception https://t.co/HVzXWSpnk4",0.996622
2,"Celebrate love this valentine without the fear of unplanned pregnancy, Weka contraceptive mpangoni, We are offering free contraceptive at a Health Clinic near you, Whatsapp us on 0709819001 https://t.co/Md56yWtrzz",0.996585
3,At Kayole health center having this wonderful dialogue on contraceptives awareness..#SautiSasa#Endteenagepregnancies#Teenagepregnanciesendswithme@Amref_Kenya @NYC_YouthVoice @fahe_k @maita_shelmith @AbukaAlfred @KombeMartha @esther_aoko @MarieStopesKe @Alvinmwangi254 https://t.co/aGLCGJz2Pm,0.993788
4,RT @WanjikuMerci: At Kayole health center having this wonderful dialogue on contraceptives awareness..#SautiSasa#Endteenagepregnancies#Teenagepregnanciesendswithme@Amref_Kenya @NYC_YouthVoice @fahe_k @maita_shelmith @AbukaAlfred @KombeMartha @esther_aoko @MarieStopesKe @Alvinmwangi254 https://t.co/aGLCGJz2Pm,0.993370
5,"An exciting and informative session on Youth, sex and contraceptives. Join the conversation @findmymethod .#FindMyMethod.@MombasaYACH @seya_youth @Epicyouthorg @KasiitaMark @Sauti_SRHR @adolescentyout1 @youth4SRHR https://t.co/RcX3CJfYHs",0.992663
6,RT @NenaNaBinti: We ensure young people have access to essential health care including access to contraceptives commodities by working in collaboration with amazing youth champions in supporting adolescents and youth to access services in their communities. #NenaNaBinti 0800211227 FREE https://t.co/yxANGx67xi,0.992036
7,"RT @lovemafrica: Celebrate love this valentine without the fear of unplanned pregnancy, Weka contraceptive mpangoni, Marie Stopes Kenya is offering free contraceptive at their Health Clinic near you, Whatsapp them on 0709819001 for more information. ~#contraception https://t.co/HVzXWSpnk4",0.990955
8,We ensure young people have access to essential health care including access to contraceptives commodities by working in collaboration with amazing youth champions in supporting adolescents and youth to access services in their communities. #NenaNaBinti 0800211227 FREE https://t.co/yxANGx67xi,0.989931
9,"RT @leujonson: An exciting and informative session on Youth, sex and contraceptives. Join the conversation @findmymethod .#FindMyMethod.@MombasaYACH @seya_youth @Epicyouthorg @KasiitaMark @Sauti_SRHR @adolescentyout1 @youth4SRHR https://t.co/RcX3CJfYHs",0.988836


In [130]:
contra_tw_neg = pd.read_sql_query('select distinct(content), sentiment from srh where content regexp "contraceptive" order by sentiment limit 10;', engine)
contra_tw_neg

,content,sentiment
0,RT @mariahakinyi11: @NAYAKenya @NMS_Kenya @MADACI_ke @lovemafrica @AYARHEP_KENYA @YouthActKE @rhnkorg @StraightTalkKe @ywli_info @CSA_Kenya @TICAH_KE @YourAuntyJane When @NMS_Kenya fails to prioritize they open window for quarks to excel this means the rising cases of teen pregnancies due to lack of contraceptives which will bring about high rate of unsafe abortion #Thebudgetwewant#BudgetForReproductiveHealth,-0.999421
1,@NAYAKenya @NMS_Kenya @MADACI_ke @lovemafrica @AYARHEP_KENYA @YouthActKE @rhnkorg @StraightTalkKe @ywli_info @CSA_Kenya @TICAH_KE @YourAuntyJane When @NMS_Kenya fails to prioritize they open window for quarks to excel this means the rising cases of teen pregnancies due to lack of contraceptives which will bring about high rate of unsafe abortion #Thebudgetwewant#BudgetForReproductiveHealth,-0.999231
2,These guys are talking about how contraceptives fail unarealize uko na mtoto akiwa 6months and that shit is scary bro,-0.998350
3,Sigh. I hope I see sex education and access to contraceptives become available to everyone in my lifetime. This is completely ridiculous https://t.co/QM17klrPxT,-0.997794
4,RT @Valenti26058615: @Zamara_fdn @DorcasWege @Rosemar21055404 @MachukiMoraa @GlobalGirlsFund @SautiSkika @youth4SRHR @SRHRforAll @BitsSrhr @StraightTalkKe @girlsvoices1 @CHOICEforYouth @Polycomgirls1 @shesthefirst With corona and the lockdowns that have been put in place a lot of young women have been unable to access contraceptives therefore affected them greatly others even getting unwanted pregnancies #COVID19ANDAGYW #ZamaraVoices,-0.997694
5,"RT @_pink_shoes: @YEMKenya @WGNRR @TICAH_KE @KELINKenya @ShankiAustine @LinKROEGER @loveablekathia @Nimechanuka @CSA_Kenya @KenyaSRHR @rhnkorg @NenaNaBinti Due to legal and socio-cultural restrictions, women and girls in Kenya and across the world are still unable to access contraceptives and safe abortion products and services. #WomensHealthMatters #SRHRisEssential",-0.997230
6,RT @mwendesusu: Sigh. I hope I see sex education and access to contraceptives become available to everyone in my lifetime. This is completely ridiculous https://t.co/QM17klrPxT,-0.996748
7,"RT @findmymethod: @YEMKenya @KELINKenya @TICAH_KE @SheLeadsAfrica @Unwomen @FemnetProg @WGNRR @ARROW_Women @wgnrr_africa @ThinkCREA @WomenSpaces @AderaBetty @NenaNaBinti @AbukaAlfred @OmutitiFredrick @GirlsWomenPower @MoH_DRMH @DrMercyHealth @MOH_Kenya 2/2 Issues tackled include mother-to-child HIV transmission, screening and treatment of cervical cancers, #GBV response hotlines, school based menstrual health management initiatives and contraceptive counselling. #WomensHealthMatters #SRHRisEssen",-0.996084
8,RT @loveablekathia: @NAYAKenya @YouthActKE @rhnkorg @StraightTalkKe @AYARHEP_KENYA @TICAH_KE @Zamara_fdn @MADACI_ke @YourAuntyJane @ReproRights @ywli_info Counselling and services for a range of modern contraceptives and access to safe abortion services and treatment of complications of unsafe abortion has been altered due to the covid-19 pandemic. #TheBudgetWeWant#BudgetForReproductiveHealth #NAYAVoices @NAYAKenya,-0.995756
9,RT @tracymokgopo: the gag is this shit has been in the works for over a decade and they still weren't sold because they have side effects but women are expected to take contraceptives despite the side effects. https://t.co/EQweuFxfef,-0.995726


- Positive: The celebration of commitment to contraceptive access. The awareness of contraceptive methods to prevent unplanned pregnancies. Informative sessions about sex and contraceptives.


- Negative: The failure of prioritising contraceptive methods budget for teen unplanned pregnancies. The lack of availability of sex education and access to contraceptives. In addition, some of the tweets with positive sentiment also have negative sentiment (for example, "unable to access contraceptives as a consequence of covid lockdowns").

### Empower (empowerment)

In [49]:
empower = pd.read_sql_query('select count(num_retweets) as "count_emporwer" from srh where content regexp "empower";', engine)
empower

,count_emporwer
0,3693


In [131]:
emp_tw_pos = pd.read_sql_query('select distinct(content), sentiment from srh where content regexp "empower" order by sentiment desc limit 10;', engine)
emp_tw_pos

,content,sentiment
0,"RT @Consumers_Kenya: @siayifabian The vision of achieving a safe, accurate, high quality, free, acceptable and voluntary family planning services and information will further aid in empowering and encouraging our women and young girls to lead better, healthier, and happier lives. #TujulishaneFamilyPlanning",0.999201
1,RT @GPtoEndFGM: #thursdayvibes Empower a girl and she will change the worldðŸ‘We @GPtoEndFGM are proud to support 9âƒ£0âƒ£3âƒ£0âƒ£2âƒ£ girls ðŸ‘§ in 3âƒ£6âƒ£2âƒ£2âƒ£ communities to gain skills and knowledge to advocate for their rights and become agents of #change in their communities in 2âƒ£0âƒ£2âƒ£0âƒ£. #FGM #EndFgm https://t.co/r6MaykHywX,0.999190
2,"@siayifabian The vision of achieving a safe, accurate, high quality, free, acceptable and voluntary family planning services and information will further aid in empowering and encouraging our women and young girls to lead better, healthier, and happier lives. #TujulishaneFamilyPlanning",0.998318
3,RT @dsw_intl: How are sexual and reproductive health and rights (SRHR) ðŸ”— to #ClimateChange? Our latest ðŸ“½ï¸ highlights how investing in SRHR is essential to empowering women and girls to lead the response to the climate crisis ðŸ“¢ and to achieving climate justice. #GenerationEquality https://t.co/rp1WbUrh4o,0.989693
4,"RT @MeTAKenya2018: Comprehensive Sexuality Education(CSE) equips adolescents with knowledge, skills, attitudes and values that will empower them to realize their health, well-being and dignity. #EndTeenPregnanciesKe #FridayThoughts #CSE #SRHR https://t.co/uRmWIFWRrY",0.982147
5,Youth Champions in #meru celebrating the 30 years of getting support from @dsw_intl @DSWKenya in Securing health and advancing #SRHR in our communities through advocacy and youth empowerment.Thank You DSW!! https://t.co/bfxNyQLasj,0.981928
6,RT @FinlandinSom: ðŸ‡«ðŸ‡® recently committed 20 MEUR to @UNFPA_SOMALIA for 2021-24. We are proud to continue joint efforts at promoting the rights and well-being of all women and girls and empowering youth leadership in peacebuilding and development. #SRHR #endFGM #FIforHRC @Ulkoministerio https://t.co/HorUEPwBBS,0.981913
7,RT @YWANKenya: Youth Champions in #meru celebrating the 30 years of getting support from @dsw_intl @DSWKenya in Securing health and advancing #SRHR in our communities through advocacy and youth empowerment.Thank You DSW!! https://t.co/bfxNyQLasj,0.980444
8,"RT @ian_indeje: #2021goals Let's focus on Empowering young people create action plans, that will ensure #YouthVoicesAreHeard at all levels by promoting #MYE through strategic &amp; SMART Advocacy that will improving knowledge, attitudes, and behaviors of youth on the ground around their #SRHR https://t.co/c1o5Pmcf9c",0.979452
9,Exploring empowerment as a way of addressing FGM is very sustainable and requires youth engagement to achieve these desired results #youthlead #YouthEndFGM,0.979114


In [132]:
emp_tw_neg = pd.read_sql_query('select distinct(content), sentiment from srh where content regexp "empower" order by sentiment limit 10;', engine)
emp_tw_neg

,content,sentiment
0,"RT @omon_oyamenda: @light_consult @boy_empowerment @MirajCecil @surei64426551 @danson_lepapa @EagleWingsOrga1 @SisterLegal @TundeDiya @7CupsTeen @AdelodunEmmanu1 @zion_ameh @TessyAnthony4 Physical indicators of Child molestationâœ“unexplained genital injuryâœ“bedwetting beyond the usual ageâœ“STIsâœ“Anal complaints (pain, bleeding)â€¢Behavioural indicatorsâœ“acute traumatic response(clingy behaviour,irritability)âœ“school problemsâœ“eating and sleeping disorders",-0.999676
1,"@Jeffngash4 @Cnx_Us @CareElimu @vinnyoma @ChildSpace1 @child_tana @boy_empowerment @EngNick3 @Faridahally2 @candybaibe170 @TwahaMaimuna Gender-based violence is the violation of rights and it takes numerous forms like sexual violence, child marriage, female genital mutilation, trafficking for sexual exploitation, female infanticide, more",-0.998375
2,RT @Alawijamalileil: I'm kinda annoyed that 8 years since we've had women reps there has been:*no policies to take care of rape and fgm victims*no free sanitary pads in schools and universities* no national women empowerment programme*No GBV support center for womenAnd they earn fat paychecks,-0.998192
3,"RT @MasesePipi: @Jeffngash4 @Cnx_Us @CareElimu @vinnyoma @ChildSpace1 @child_tana @boy_empowerment @EngNick3 @Faridahally2 @candybaibe170 @TwahaMaimuna Gender-based violence is the violation of rights and it takes numerous forms like sexual violence, child marriage, female genital mutilation, trafficking for sexual exploitation, female infanticide, more",-0.998102
4,RT @Tomma_kiteto: @stagemediaarts @Real_Nixon_ @J_Nyando @YourAuntyJane @BungomaCountyKe @CWangamati_ @safe2choose @BYCke_ @YouthActKE @CboHype @MSIchoices @DreamAchieversk @BungomaRmncah @DeclaresKenya @AYGKenya @reprodrive This the gobal obligation to empower girls and women to access services on prevention treatment and care without travelling a long distance or wasting time#SRHR,-0.997434
5,RT @Bariandos: @DeclaresGroupKe @DSWKenya @hivosrosa @VoiceandAgency1 @PublicPathways @FP2030Global @amplifyfund @unwomenkenya @icrhk_official @gchallenges @CanHCKenya @tendasasa @YouthActKE @AfyaJadili @hivos @WorldVisionKE @voicetweetz @YouthAntiFGMKe @safecommunity4 @PwaniYouth @DreamAchieversk @UNODC Women lack empowerment to speak about their sexual reproductive health issues in the community @hivossa #IWD2021 #GenerationEquality #IChooseToChallenge,-0.987328
6,"RT @DSWKenya: @Nyandarua_Youth @NyandaruaCG018 @NyandaruaCA Access to #SRHinformation through age-appropriate #ComprehensiveSexualityEducation empowers adolescents and the youth to overcome challenges, including unintended teenage pregnancy, unsafe abortions and new HIV infections and #IamforCSE #SRHIsEssential #NiwekeKwaBudget",-0.983693
7,@Af_Rika @leujonson @taliacarol_ @ManaseNjagi @raccemaurice @EnockKiptanui @Malcandi @Amref_Kenya @YouthActKE @Alvinmwangi254 @ShankiAustine @MyDawaApp Women and girls are faced by a number of challenges which include 1.Gender Based Violence2.women fall victims to harmful cultural practices such as FGM and early/forced marriages 3.Economic disempowerment which makes them more vulnerable #ChallengeAccepted,-0.980371
8,Why Champion for abortion? Does it mean it's the biggest gap between our girls and empowerment? Is abortion a way of empowerment?@annmtave#AbortionIsMurder,-0.978703
9,"@Nyandarua_Youth @NyandaruaCG018 @NyandaruaCA Access to #SRHinformation through age-appropriate #ComprehensiveSexualityEducation empowers adolescents and the youth to overcome challenges, including unintended teenage pregnancy, unsafe abortions and new HIV infections and #IamforCSE #SRHIsEssential #NiwekeKwaBudget",-0.977417


- Positive: The defence of safe, high quality, informative and voluntary family planning services. The need for female empowerment to change the world. The empowerment of youth leadership in peacebuilding and development. 


- Negative: The condemnation of child molestation, gender-based violation, child marriage, FGM, and sex trafficking, among others.

### Politic

In [55]:
politic = pd.read_sql_query('select count(num_retweets) as "count_politic" from srh where content regexp "politic";', engine)
politic

,count_politic
0,1218


In [135]:
politic_tw_pos = pd.read_sql_query('select distinct(content), sentiment from srh where content regexp "politic" order by sentiment desc limit 10;', engine)
politic_tw_pos

,content,sentiment
0,RT @NankaliM: Congratulations to #kenya. It is this kind of political will and commitment that is required to eliminate FGM. Thank you for leading the world on this agenda! https://t.co/DWKXA9XwZy,0.998072
1,"RT @Ohta_Ryota: @DCI_Kenya The reason 'GWATA NDAI' is gaining confidence and strength is because Kikuyu musicians, radio presenters, politicians and businessmen are using it for their own gains and even bank rolling it. FGM is being propagated by the group among other things",0.980080
2,"RT @loveablekathia: Providing an essential package of SRHR interventions with a life course approach,applying equity in access to quality care without discrimination, &amp; accountability across implementation needs a political will. #BudgetForReproductiveHealth #TheBudgetWeWant #NAYAVoices @NAYAKenya",0.976299
3,RT @obianuju: Today is the perfect day to repost this excellent video of Cardinal Arinze speaking about abortion-supporting politiciansâ€¦ https://t.co/GkorvC0bU1,0.975491
4,"Providing an essential package of SRHR interventions with a life course approach,applying equity in access to quality care without discrimination, &amp; accountability across implementation needs a political will. #BudgetForReproductiveHealth #TheBudgetWeWant #NAYAVoices @NAYAKenya",0.970496
5,"@DCI_Kenya The reason 'GWATA NDAI' is gaining confidence and strength is because Kikuyu musicians, radio presenters, politicians and businessmen are using it for their own gains and even bank rolling it. FGM is being propagated by the group among other things",0.963749
6,"RT @IpasOrg: Great news in Europe! The EU Parliament voted to prioritize #SRHR issues including #CSE, contraception and abortion. Great to see more politicians committing to promote &amp; protect #reprohealth &amp; #reprorights. Thanks @fred_matic for your hard work. https://t.co/YuqBy9Pcbg",0.949146
7,"RT @NCIKenya: Today, 8th March; Is International Women's Day: A day we celebrate the social, economic, cultural and political achievements of women and rally for women's equal rights including sexual and reproductive health rights. #internationalwomensday2021 #InternationalWomensDay",0.948262
8,RT @SadearH: As we celebrate #IWD2021 meet Hon Hussein Dado CAS interior saying #EndFGM #PoliticalGoodWill is key for us to celebrate Zero FGM by 2022 in Kenya. @StateHouseKenya @LisaStadelbauer @AWRA_org @OlajideDemola @daktari1 @lolojuberna @JaneMarriottUK @MenEngageAfrica @GMCEndFGM https://t.co/idvmNMieWp,0.935888
9,"RT @liz_sugg: On #WorldHealthDay, let's remember the importance of sexual &amp; reproductive health &amp; rights - critical to achieve ðŸŒ ambitions to truly #BuildBackBetter after COVID-19.For real health equity, women &amp; girls need political &amp; financial support to access comprehensive #SRHR services",0.930406


In [136]:
politic_tw_neg = pd.read_sql_query('select distinct(content), sentiment from srh where content regexp "politic" order by sentiment limit 10;', engine)
politic_tw_neg

,content,sentiment
0,"@jayp__morgan @Hdanielzxs They are not the only ones who have died in Kenya today,my grandy,who fought world war 2,died,covid,hunger,abortion, suicide,killed hundreds today,only the socio-political elite are sad,not even sad,the death of a millionaire is the happiest news to the relatives and competitors",-0.999098
1,"RT @nahashon87: KOT A: Comfortable with breaking news e.g. nani amekufa or feel good opinions e.g. politicians are badKOT B: Comfortable with sports talk, sex talk, &amp; chaiKOT C: Comfortable with insults as long as they know your stand so that they can insult you or support you insulting others",-0.997012
2,If we really fought rape the way we're fighting FGM. It would really be successful.There is a way rape is being fought nowadays and it involves a lot of damn politics. It's boring.,-0.994121
3,"@ReyannaRice @Politic85738300 @GodfreydeBouil1 @LJ_Milone @USCCB Of course it makes you very sad that innocent babies are yanked from their mothers wombs and killed. If we can eliminate all abortion except those due to rape and incest, we will have saved so many lives &amp; it would be a step in the right direction as we eliminate rape &amp; incest.",-0.986005
4,"RT @ongondo_: As politicians tend to shy away from the sensitive topic of FGM, the responsibility falls on charities to do the work, it's sad that politicians we have chosen have left innocent kenyans to suffer #MenEndFGM https://t.co/OsRbkkOWe7",-0.982949
5,"RT @DavidNdii: @wmnjoya That aside, I am intrigued by people who routinely engage in the crudest most bigoted verbal wars on foreskins, abortion, atheism, homosexuality and even physical fights for politicians, are mortified by a harmless adversarial intellectual debate.7/",-0.981988
6,"RT @Sheiske2: @Zamara_fdn @cathy_osita @WendyObel @Lizmwan56479680 @ruraltoglobal @polycomdev @Nairobits @AMPLIFY_girls @YStakeholders @YWLC_Kenya @MoyoteKenya @yc_kenya @NAYAKenya @womenslink Women and girls' sexual and reproductive healthcare needs increase whenever outside factors change,(facors like political atmosphere, environmental changes-including Covid19). Such services have been disrupted, dramatically affecting how women work. #COVID19andLivelihoods",-0.981018
7,"As politicians tend to shy away from the sensitive topic of FGM, the responsibility falls on charities to do the work, it's sad that politicians we have chosen have left innocent kenyans to suffer #MenEndFGM https://t.co/OsRbkkOWe7",-0.977153
8,RT @terrikibiriti: NO POLITICIAN SHOULD HAVE BEEN ALLOWED AT THIS FUNERAL.Y'ALL AS USELESS AS THE CONDOM DISPENSER AT THE VATICAN https://t.co/TZTliomJ9Y,-0.975394
9,RT @beverly_nkirote: @StateHouseKenya @MOH_Kenya continues to fail Millions of women and girls seeking to access comprehensive SRHR information and services including access to safe and legal Abortion due to lack of political goodwill in implementing existing policies #AbortionIsHealthCareKe https://t.co/l42W8RK0k0,-0.973793


- Positive: Political and cultural commitment to eliminate FGM. The implementation of policies in favour of quality care without discrimination. The EU Parliament voted to prioritise reproductive rights (not geographically related).


- Negative: The demand to the politicians for equality in fighting rape and FGM. The condemnation for the lack of initiative from the politicians for fighting FGM or the demand for more funds for sexual and reproductive healthcare. 

### Violen (violent, violence)

In [97]:
violen = pd.read_sql_query('select count(num_retweets) as "count_violen" from srh where content regexp "violen";', engine)
violen

,count_violen
0,4148


In [137]:
viol_tw_pos = pd.read_sql_query('select distinct(content), sentiment from srh where content regexp "violen" order by sentiment desc limit 10;', engine)
viol_tw_pos

,content,sentiment
0,"RT @ippfen: ""...Good sexuality education encourages respectful and healthy relationships, promotes respectful diversity, human and fundamental rights, it helps combat gender stereotypes, violence and abuse against children &amp; women and discrimination."" @helenadalli ðŸŸ https://t.co/k2zBLYegyT https://t.co/vR7AsSgzT7",0.991046
1,RT @KivuvaniMwikali: Congratulations @PlanKenya on the New Country Strategy 2021-2026. The @KenyaSRHR is proud to be your partner and friend as we work towards ending teenage pregnancies and all forms of gender based violence against adolescent girls and young women. https://t.co/wCdfGSDybh,0.979387
2,"@Kamamifay Sexual health requires a positive and respectful approach to sexuality and sexual relationships, as well as the possibility of having pleasurable and safe sexual experiences, free of coercion, discrimination and violence. #TujulishaneReportLaunch",0.979048
3,"RT @ippfen: ðŸ‘ðŸ¼ðŸ‡ªðŸ‡º Thank you Commissioner @helenadalli for affirming that relationship and sexuality education should be an essential component of the future EU Directive on Gender-Based Violence. ðŸ¤©Listen to our podcast, Gender &amp; The Union, for more https://t.co/k2zBLYegyT#UnionOfEquality https://t.co/6jrQmEp237",0.976966
4,"RT @Consumers_Kenya: @Kamamifay Sexual health requires a positive and respectful approach to sexuality and sexual relationships, as well as the possibility of having pleasurable and safe sexual experiences, free of coercion, discrimination and violence. #TujulishaneReportLaunch",0.973999
5,"RT @1girldreams: Sexual health demands positive and respectful approaches to sexuality and sexual relationships, and the possibility of having pleasurable and safe sexual experiences,free from coercion,discrimination and violence. @amwaafrika @sexualhealthday @Leah_Wanjie #turniton #wshd2021 https://t.co/gPrqi0ORjx",0.973738
6,"RT @Sauti_SRHR: Sexual health demands positive and respectful approaches to sexuality and sexual relationships, and the possibility of having pleasurable and safe sexual experiences,free from coercion,discrimination and violence. @amwaafrika @sexualhealthday @Leah_Wanjie #turniton #wshd2021 https://t.co/P9kHw6Ql4f",0.973738
7,"RT @AWRA_org: Happy #DayOfTheAfricanChild!âž¡ï¸ This beautiful ðŸ“¸ was taken by founding member @CDomtila, who everyday works to ensure that every girl child lives a life free of violence, FGM, and early marriage. Childhood is a beautiful experience that must be held sacred. https://t.co/SdxYOhFRxl",0.971716
8,So excited to be a part of the @WO50FF family and that Becky trusted me to make this film of a piece of her story with FGM. https://t.co/3NNAHrJJO7 #FGM #UNFPA #EndViolenceAgainstWomen #HumanRights #SDGs,0.969443
9,"SH requires a positive &amp; respectful approach to sexuality &amp; sexual relationships, as well as having pleasurable and safe sexual experiences, free of coercion, discrimination &amp; violence. For it to be attained the sexual rights of persons must be respected &amp; protected #WhenSexHurts",0.965637


In [138]:
viol_tw_neg = pd.read_sql_query('select distinct(content), sentiment from srh where content regexp "violen" order by sentiment limit 10;', engine)
viol_tw_neg

,content,sentiment
0,"RT @Mukami_Mungai: @KegediL @Alvinmwangi254 @firdausamo @G_MachelTrust @UNFPAKen @equalitynow @UNFPA_ESARO @YouthAmbSRHR @SRHRAfricaTrust @SRHRforAll @MoH_DRMH @MOH_Kenya @DrMercyHealth @DrMercyKorir @JulittaOnabanjo @IYAFPKenya @ConnectAID_int @KenyaSRHR @FemnetProg @crawntrustkenya @feminstyle -Address gender inequalities, power imbalances and male dominance in order for women to meaningfully claim and enjoy freedom from violence, abuse, coercion and discrimination. #MaputoProtocol #GlobalGoal",-0.999781
1,"RT @beverly_nkirote: @NAYAKenya @NMS_Kenya @MADACI_ke @lovemafrica @AYARHEP_KENYA @YouthActKE @rhnkorg @StraightTalkKe @ywli_info @CSA_Kenya @TICAH_KE @YourAuntyJane When @NMS_Kenya fails to prioritize the reproductive health of women and girls we then have increased cases of unsafe abortions , unintended pregnancies, increased cases of sexual and gender violence , increased cases of FGM and Child marriages #TheBudgetWeWant @MOH_Kenya",-0.999685
2,"@KegediL @Alvinmwangi254 @firdausamo @G_MachelTrust @UNFPAKen @equalitynow @UNFPA_ESARO @YouthAmbSRHR @SRHRAfricaTrust @SRHRforAll @MoH_DRMH @MOH_Kenya @DrMercyHealth @DrMercyKorir @JulittaOnabanjo @IYAFPKenya @ConnectAID_int @KenyaSRHR @FemnetProg @crawntrustkenya @feminstyle -Address gender inequalities, power imbalances and male dominance in order for women to meaningfully claim and enjoy freedom from violence, abuse, coercion and discrimination. #MaputoProtocol #GlobalGoals #SRHR #ActForEqua",-0.999563
3,"@NAYAKenya @NMS_Kenya @MADACI_ke @lovemafrica @AYARHEP_KENYA @YouthActKE @rhnkorg @StraightTalkKe @ywli_info @CSA_Kenya @TICAH_KE @YourAuntyJane When @NMS_Kenya fails to prioritize the reproductive health of women and girls we then have increased cases of unsafe abortions , unintended pregnancies, increased cases of sexual and gender violence , increased cases of FGM and Child marriages #TheBudgetWeWant @MOH_Kenya",-0.999500
4,"RT @Thalma69801073: @ragz2beechez @Zamara_fdn Alot of low income earners have been rendered jobless due to the pandemic, we have had increasing cases on gender based violence because victims are now stuck with their predators,schools have been shut and students put at risk of teenage pregnancies.#COVID19andLivelihoods",-0.998759
5,@Kcyg001 @TISAKenya @a_nastehe @sharonjessey @AnjiaSaraphina @GlobalFundWomen @MombasaTalk @wardababu001 @RitahAnindo @DSWKenya Gender based violence increases the risk of unwanted and unintended pregnancy being that some women at that particular time are not able to access family planning services especially during this pandemic,-0.998541
6,"RT @bennetowuonda: @Midrifthurinet @karuga @WillieOeba @KMapenai @ShankiAustine GBV includes:Sexual violence, including rape, sexual abuse, sexual exploitation and forced prostitutionDomestic violenceForced and early marriageHarmful traditional practices such as female genital mutilation, honour crimes, widow inheritanceTrafficking#MH4childrenandYouth",-0.998501
7,"@Jeffngash4 @Cnx_Us @CareElimu @vinnyoma @ChildSpace1 @child_tana @boy_empowerment @EngNick3 @Faridahally2 @candybaibe170 @TwahaMaimuna Gender-based violence is the violation of rights and it takes numerous forms like sexual violence, child marriage, female genital mutilation, trafficking for sexual exploitation, female infanticide, more",-0.998375
8,"RT @MasesePipi: @Jeffngash4 @Cnx_Us @CareElimu @vinnyoma @ChildSpace1 @child_tana @boy_empowerment @EngNick3 @Faridahally2 @candybaibe170 @TwahaMaimuna Gender-based violence is the violation of rights and it takes numerous forms like sexual violence, child marriage, female genital mutilation, trafficking for sexual exploitation, female infanticide, more",-0.998102
9,"RT @NdoloOliver: @SonofTetu @elfafrica1 @Gvrckenya @OpenSocietyEA Victims ofÂ violenceÂ can suffer sexual and reproductiveÂ health consequences, including forced and unwanted pregnancies, unsafe abortions, traumatic fistula, sexually transmitted infections including HIV, and even death.#EndGB

- Positive: The promotion of proper sexual education and fundamental rights against violence and abuse of women and children. The celebration of the African Child Day.


- Negative: The condemnation for gender inequalities to enjoy freedom from violence, abuse, coercion and discrimination. The fight against gender inequalities, power imbalances and male dominance, so women can enjoy freedom from violence and discrimination. The demand for the abolition of traditional practices such as FGM or widow inheritance trafficking. 

### HIV

In [102]:
hiv = pd.read_sql_query('select count(num_retweets) as "count_hiv" from srh where content regexp "hiv";', engine)
hiv          # adding the search for 'aids', the total number is 11.261 (9483 + 1778)

,count_hiv
0,9483


In [140]:
hiv_pos = pd.read_sql_query('select distinct(content), sentiment from srh where content regexp "hiv" order by sentiment desc limit 10;', engine)
hiv_pos

,content,sentiment
0,"RT @ruraltoglobal: @ian_indeje @KuzaSME @HerVoiceFund @Yplus_Global @ypluskenya @ayan_keny @weteckenya @fhokmyc @CboChana @MombasaYACH @totocentre @CboGirls @path_youth_org @moh They can advocate for the facilitation of affordable, accessible, acceptable and quality health care services including reproductive health care, emergency services, family planning, HIV and AIDS service for women and men, girls and boys. #AGYW #HERVoice @HerVoiceFund @CafeSRHR",0.989273
1,"@NAYAKenya @HeadMediaDesign @hivosrosa @CSA_Kenya @rhnkorg @StraightTalkKe @YouthActKE @YourAuntyJane @beverly_nkirote @YEMKenya @GEM_Africa @TICAH_KE @ywli_info Hello, we need partnership with @KisiiCountyGov for SRHR,SGBV programming for youth,please get in touch. Thanks.",0.988088
2,"We will also develop an array of creative and innovative digital communication approaches to mobilize communities, catalyze dialogue, reduce risky behaviors, and create market-based strategies to create innovative solutions that will improve knowledge about HIV, STIs,ðŸ˜‚ and STDs. https://t.co/ETP4XFQUPb",0.987860
3,"Congratulations to the amazing @OfficialJMbugua and @InuaDada foundation for winning a global award from the World Association for Sexual Health at the 25th Congress of the World Association for Sexual Health, funded by Hivos. I celebrate youðŸ‘. Cheers to more wins! https://t.co/u4LhbzfM3c",0.986406
4,RT @ahiveco: Congratulations on your recognition for excellence and innovation in Sexuality Education in Africa. ðŸ‘ðŸ¾ðŸ‘ðŸ¾@OfficialJMbugua!@InuaDada #sexualhealthmatters #menstruationmatters #genderequality #srhr https://t.co/Qcsld4Gwj1,0.986079
5,"RT @KateVorley: Congratulations to the amazing @OfficialJMbugua and @InuaDada foundation for winning a global award from the World Association for Sexual Health at the 25th Congress of the World Association for Sexual Health, funded by Hivos. I celebrate youðŸ‘. Cheers to more wins! https://t.co/u4LhbzfM3c",0.986037
6,"RT @EnCompass_World: Through our many years of working in global health, we have supported improved health outcomes related to HIV, malaria, tuberculosis, family planning, sexual and reproductive health, maternal, newborn, and child health, nutrition, and adolescent health. https://t.co/rZSsDHjEx0",0.984860
7,"RT @NAYAKenya: We're very excited to announce the winners of a #HIV prevention and #SRHR digital project that NAYA worked on in collaboration with #GenEndIt, @HeadMediaDesign (the University of Design and Applied Arts of Geneva), and @hivosrosa. #AmaniMyFriend https://t.co/yRZE3n0nNb",0.982529
8,"RT @_KarenHospital: We are proud to announce that we now have a PCR GeneXpert machine.This means that we will be able to carry out the following tests inhouse with guaranteed faster TAT and highly accurate results:- COVID-19, TB GeneXpert, HIV Viral Load, CT/NG (STIs). @DrBettyGikonyo @TKH_CEO https://t.co/Kud31rXqYq",0.979296
9,"RT @progress_md: It's love season again but do you know your HIV/AIDS status?. Ensure you get tested and use a condom to protect yourself.We can love and live in great health at the same time, if we love smartly. https://t.co/gQWAzW0bTx",0.977449


In [141]:
hiv_neg = pd.read_sql_query('select distinct(content), sentiment from srh where content regexp "hiv" order by sentiment limit 10;', engine)
hiv_neg

,content,sentiment
0,"RT @MADACI_ke: @NAYAKenya @NMS_Kenya @lovemafrica @AYARHEP_KENYA @YouthActKE @rhnkorg @StraightTalkKe @ywli_info @CSA_Kenya @TICAH_KE @YourAuntyJane Q4:When NMS fails to #BudgetForReproductiveHealth women and girls will be denied access to Sexual and Reproductive Health services and information hence putting the at risk of teenage pregnancies, unsafe abortions and STIs and HIV infections.#TheBudgetWeWant",-0.999846
1,"@NAYAKenya @NMS_Kenya @lovemafrica @AYARHEP_KENYA @YouthActKE @rhnkorg @StraightTalkKe @ywli_info @CSA_Kenya @TICAH_KE @YourAuntyJane Q4:When NMS fails to #BudgetForReproductiveHealth women and girls will be denied access to Sexual and Reproductive Health services and information hence putting the at risk of teenage pregnancies, unsafe abortions and STIs and HIV infections.#TheBudgetWeWant",-0.999790
2,"RT @kajuju_nthiana: @NAYAKenya @rhnkorg @ywli_info @AYARHEP_KENYA @StraightTalkKe @YourAuntyJane @MADACI_ke @NenaNaBinti @FP2030Global @KombeMartha @Alvinmwangi254 @GirlsWomenPower @esther_aoko @Eve_odhis @YouthActKE @TICAH_KE @AbukaAlfred @MokayaKelvinOb @DreamAchieversk Safe sex means sexual activity in which one takes precaution and reduces risk of hiv, STIs and other sexually transmitted diseases #WhenSexHurts #FP4All #NAYAVOICES",-0.999612
3,"RT @bettymtuweta: @DeclaresGroupKe @DSWKenya @hivosrosa @VoiceandAgency1 @PublicPathways @FP2030Global @amplifyfund @unwomenkenya @icrhk_official @gchallenges @CanHCKenya @tendasasa @YouthActKE @AfyaJadili @hivos @WorldVisionKE @voicetweetz @YouthAntiFGMKe @safecommunity4 @PwaniYouth @DreamAchieversk @UNODC Teenage pregnancies hindering young girls to acess education, inability to acess MHM products and sanitary products #IChooseToChallenge #IWD2021 @Thatguyjeffar @PabloMnangat",-0.999566
4,RT @corrasyne2: @NAYAKenya @NMS_Kenya @MADACI_ke @lovemafrica @AYARHEP_KENYA @YouthActKE @rhnkorg @StraightTalkKe @ywli_info @CSA_Kenya @TICAH_KE @YourAuntyJane When @NMS_Kenya fails to #BudgetForReproductiveHealth there's a rise in teenage pregnancies leading to the cases of unsafe abortion (which has terrible effects) and also there will be cases of higher spread of HIV@CSA_Kenya #TheBudgetWeWant #NAYAVoices,-0.999443
5,RT @CheechWinnie: @DeclaresGroupKe @DSWKenya @hivosrosa @VoiceandAgency1 @PublicPathways @FP2030Global @amplifyfund @unwomenkenya @icrhk_official @gchallenges @CanHCKenya @tendasasa @YouthActKE @AfyaJadili @hivos @WorldVisionKE @voicetweetz @YouthAntiFGMKe @safecommunity4 @PwaniYouth @DreamAchieversk @UNODC #IWD2021 maternal death is one of the issue affecting women and girls on SRH and this is caused due lack of access to safe and affordable health services. #IChooseToChallenge #GenerationEqual,-0.999360
6,RT @Bariandos: @DeclaresGroupKe @DSWKenya @hivosrosa @VoiceandAgency1 @PublicPathways @FP2030Global @amplifyfund @unwomenkenya @icrhk_official @gchallenges @CanHCKenya @tendasasa @YouthActKE @AfyaJadili @hivos @WorldVisionKE @voicetweetz @YouthAntiFGMKe @safecommunity4 @PwaniYouth @DreamAchieversk @UNODC Women in rural areas are the most affected in violation of sexual reproductive health rights due to unsupportive environment @OptionsKe @CanHCKenya #IWD2021 #GenerationEquality #IChooseToChallen,-0.999306
7,"@NAYAKenya @rhnkorg @ywli_info @AYARHEP_KENYA @StraightTalkKe @YourAuntyJane @MADACI_ke @NenaNaBinti @FP2030Global @KombeMartha @Alvinmwangi254 @GirlsWomenPower @esther_aoko @Eve_odhis @YouthActKE @TICAH_KE @AbukaAlfred @MokayaKelvinOb @DreamAchieversk Safe sex means sexual activity in which one takes precaution and reduces risk of hiv, STIs and other sexually transmitted diseases #WhenSexHurts #FP4All #NAYAVOICES",-0.999294
8,"@DeclaresGroupKe @DSWKenya @hivosrosa @VoiceandAgency1 @PublicPathways @FP2030Global @amplifyfund @unwomenkenya @icrhk_official @gchallenges @CanHCKenya @tendasasa @YouthActKE @AfyaJadili @hivos @WorldVisionKE @voicetweetz @YouthAntiFGMKe @safecommunity4 @PwaniYouth @DreamAchieversk @UNODC Teenage pregnancies hinde

- Positive: The advocation for accessible and quality health care, family planning and HIV services for the population. The development of creative and innovative digital communication to improve the knowledge about HIV and STDs. The recognition for excellence and innovation in sexuality education in Africa.


- Negative: The demand for more budget for reproductive health to prevent denied access to those services. The need for more information on the risk of teenage pregnancies. The denounce of women in rural areas are the most affected by violation of sexual reproductive health rights. The demand for safe sex activity to prevent sexually transmitted diseases.

### Pregnan (pregnancy, pregnant)

In [110]:
pregnan = pd.read_sql_query('select count(num_retweets) as "count_pregnan" from srh where content regexp "pregnan";', engine)
pregnan

,count_pregnan
0,24037


In [143]:
preg_pos = pd.read_sql_query('select distinct(content), sentiment from srh where content regexp "pregnan" order by sentiment desc limit 10;', engine)
preg_pos

,content,sentiment
0,"FertipilPlus for Menâž¡ï¸ Boosts and improves sperm countâž¡ï¸ Improves sperm mobility and morphologyâž¡ï¸ Improves spermatogenesisâž¡ï¸ Boosts natural reproductive health and sexual functionKshs 2,495/- Tel. 0794 429 991 #sperm #spermcount #fertipil #fertipilplusformen #pregnant https://t.co/ULFxePtyrm",0.997644
1,"FertipilPlus for Menâž¡ï¸ Boosts and improves sperm countâž¡ï¸ Improves sperm mobility and morphologyâž¡ï¸ Improves spermatogenesisâž¡ï¸ Boosts natural reproductive health and sexual functionKshs 2,495/- Tel. 0794 429 991#sperm #spermcount #fertipil #fertipilplusformen #pregnant https://t.co/4zsI5IVTMx",0.997644
2,"FertipilPlus for Menâž¡ï¸ Boosts and improves sperm countâž¡ï¸ Improves sperm mobility and morphologyâž¡ï¸ Improves spermatogenesisâž¡ï¸ Boosts natural reproductive health and sexual functionKshs 2,495/- Tel. 0794 429 991#sperm #spermcount #fertipil #fertipilplusformen #pregnant https://t.co/QJZgg0rVMv",0.997644
3,"FertipilPlus for Menâž¡ï¸ Boosts and improves sperm countâž¡ï¸ Improves sperm mobility and morphologyâž¡ï¸ Improves spermatogenesisâž¡ï¸ Boosts natural reproductive health and sexual functionKshs 2,495/- Tel. 0794 429 991#sperm #spermcount #fertipil #fertipilplusformen #pregnant https://t.co/UJaMyPXJv3",0.997644
4,"FertipilPlus for Menâž¡ï¸ Boosts and improves sperm countâž¡ï¸ Improves sperm mobility and morphologyâž¡ï¸ Improves spermatogenesisâž¡ï¸ Boosts natural reproductive health and sexual functionKshs 2,495/- Tel. 0794 429 991#sperm #spermcount #fertipil #fertipilplusformen #pregnant https://t.co/YkIctxyR3Y",0.997644
5,"Celebrate love this valentine without the fear of unplanned pregnancy, Weka contraceptive mpangoni, Marie Stopes Kenya is offering free contraceptive at their Health Clinic near you, Whatsapp them on 0709819001 for more information. ~#contraception https://t.co/HVzXWSpnk4",0.996622
6,"Celebrate love this valentine without the fear of unplanned pregnancy, Weka contraceptive mpangoni, We are offering free contraceptive at a Health Clinic near you, Whatsapp us on 0709819001 https://t.co/Md56yWtrzz",0.996585
7,RT @fertilityshoppe: Available at @MyDawaApp FertipilPlus for Menâž¡ï¸ Boosts and improves sperm countâž¡ï¸ Improves sperm mobility and morphologyâž¡ï¸ Improves spermatogenesisâž¡ï¸ Boosts natural reproductive health and sexual function#sperm #spermcount #fertipil #fertipilplusformen #pregnant https://t.co/vWxeR7qlCM,0.996231
8,Thank you all for a fruitful and engaging discussion let's fight together against teenage pregnancy #ajendaniASHRH,0.995531
9,RT @YouthActKE: We're honored and excited to have been part of today's influencer brunch with @better4kenya. Here's to building partnerships geared towards ending teenage pregnancies in Kenya. #SautiSasa https://t.co/n5cyLy9v1W,0.995101


In [144]:
preg_neg = pd.read_sql_query('select distinct(content), sentiment from srh where content regexp "pregnan" order by sentiment limit 10;', engine)
preg_neg

,content,sentiment
0,"RT @maita_shelmith: @NAYAKenya @NMS_Kenya @MADACI_ke @lovemafrica @AYARHEP_KENYA @YouthActKE @rhnkorg @StraightTalkKe @ywli_info @CSA_Kenya @TICAH_KE @YourAuntyJane When @NMS_Kenya fails to prioritize SRHR and budgetary allocations for SRH ,it encourages the continuation of quack abortion services,increases the alarming numbers of unwanted teen pregnancies ,unsafe abortions and even death .#TheBudgetWeWant #BudgetForReproductiveHealth",-0.999942
1,"@NAYAKenya @NMS_Kenya @MADACI_ke @lovemafrica @AYARHEP_KENYA @YouthActKE @rhnkorg @StraightTalkKe @ywli_info @CSA_Kenya @TICAH_KE @YourAuntyJane When @NMS_Kenya fails to prioritize SRHR and budgetary allocations for SRH ,it encourages the continuation of quack abortion services,increases the alarming numbers of unwanted teen pregnancies ,unsafe abortions and even death .#TheBudgetWeWant #BudgetForReproductiveHealth",-0.999906
2,"RT @MADACI_ke: @NAYAKenya @NMS_Kenya @lovemafrica @AYARHEP_KENYA @YouthActKE @rhnkorg @StraightTalkKe @ywli_info @CSA_Kenya @TICAH_KE @YourAuntyJane Q4:When NMS fails to #BudgetForReproductiveHealth women and girls will be denied access to Sexual and Reproductive Health services and information hence putting the at risk of teenage pregnancies, unsafe abortions and STIs and HIV infections.#TheBudgetWeWant",-0.999846
3,"@NAYAKenya @NMS_Kenya @lovemafrica @AYARHEP_KENYA @YouthActKE @rhnkorg @StraightTalkKe @ywli_info @CSA_Kenya @TICAH_KE @YourAuntyJane Q4:When NMS fails to #BudgetForReproductiveHealth women and girls will be denied access to Sexual and Reproductive Health services and information hence putting the at risk of teenage pregnancies, unsafe abortions and STIs and HIV infections.#TheBudgetWeWant",-0.999790
4,"RT @beverly_nkirote: @NAYAKenya @NMS_Kenya @MADACI_ke @lovemafrica @AYARHEP_KENYA @YouthActKE @rhnkorg @StraightTalkKe @ywli_info @CSA_Kenya @TICAH_KE @YourAuntyJane When @NMS_Kenya fails to prioritize the reproductive health of women and girls we then have increased cases of unsafe abortions , unintended pregnancies, increased cases of sexual and gender violence , increased cases of FGM and Child marriages #TheBudgetWeWant @MOH_Kenya",-0.999685
5,"RT @bettymtuweta: @DeclaresGroupKe @DSWKenya @hivosrosa @VoiceandAgency1 @PublicPathways @FP2030Global @amplifyfund @unwomenkenya @icrhk_official @gchallenges @CanHCKenya @tendasasa @YouthActKE @AfyaJadili @hivos @WorldVisionKE @voicetweetz @YouthAntiFGMKe @safecommunity4 @PwaniYouth @DreamAchieversk @UNODC Teenage pregnancies hindering young girls to acess education, inability to acess MHM products and sanitary products #IChooseToChallenge #IWD2021 @Thatguyjeffar @PabloMnangat",-0.999566
6,"@NAYAKenya @NMS_Kenya @MADACI_ke @lovemafrica @AYARHEP_KENYA @YouthActKE @rhnkorg @StraightTalkKe @ywli_info @CSA_Kenya @TICAH_KE @YourAuntyJane When @NMS_Kenya fails to prioritize the reproductive health of women and girls we then have increased cases of unsafe abortions , unintended pregnancies, increased cases of sexual and gender violence , increased cases of FGM and Child marriages #TheBudgetWeWant @MOH_Kenya",-0.999500
7,RT @corrasyne2: @NAYAKenya @NMS_Kenya @MADACI_ke @lovemafrica @AYARHEP_KENYA @YouthActKE @rhnkorg @StraightTalkKe @ywli_info @CSA_Kenya @TICAH_KE @YourAuntyJane When @NMS_Kenya fails to #BudgetForReproductiveHealth there's a rise in teenage pregnancies leading to the cases of unsafe abortion (which has terrible effects) and also there will be cases of higher spread of HIV@CSA_Kenya #TheBudgetWeWant #NAYAVoices,-0.999443
8,RT @mariahakinyi11: @NAYAKenya @NMS_Kenya @MADACI_ke @lovemafrica @AYARHEP_KENYA @YouthActKE @rhnkorg @StraightTalkKe @ywli_info @CSA_Kenya @TICAH_KE @YourAuntyJane When @NMS_Kenya fails to prioritize they open window for quarks to excel this means the rising cases of teen pregnancies due to lack of contraceptives which will bring about high rate of unsafe abortion #Thebudgetwewant#BudgetForReproductiveHealth,-0.999421
9,@StretchersYouth @djleniumkenya

- Positive: Advertisement for fertility products (not related specifically to the topic). The give of thanks for engaging in discussions against teenage pregnancy.


- Negative: The dennounce for lack of budget to access to sexual and reproductive health services, information or access to health services during Covid lockdowns. 